# Generating Recidivism Data

The purpose of this notebook is to transform and clean data from North Carolina Department of Corrections into a dataset for predicting recidivsm of individual inmates. The scripts to download the raw, publically available data can be found in this repository. This notebook is likely best run on a server with suitable memory, as the data is fairly large. At the end, this exports a pickle of a pandas DF. For using in another script, the pickle is highly recommended, as import time and disk space is much lower. Can be changed to export a CSV for cross compatibility.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

# Load In Data

### Court commitments data 
- At the individual commitment level (person and commitment IDs). We select a few columns based on what might be useful in predicting recidivism, what can be gotten elsewhere, and what doesn't contain too many NaN's.

In [2]:
court_commit = pd.read_csv('data/preprocessed/OFNT3BB1.csv')

/mnt/data/users/lhinkson/.nctriage/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
court_commit.tail(5)

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,COMMITTED_LAST_NAME,COMMITTED_FIRST_NAME,COMMITTED_MIDDLE_NAME,COMMITTED_NAME_SUFFIX,OFFENDER_ADMISSION/INTAKE_DATE,P&P_CASE_INTAKE_DATE,INMATE_COMMITMENT_STATUS_FLAG,COMMITMENT_STATUS_DATE,EARLIEST_SENTENCE_EFFECTIVE_DT,NEW_PERIOD_OF_INCARCERATION_FL,MOST_SERIOUS_OFFENSE_CODE,CO_OF_CONV_MOST_SERIOUS_OFFNSE,TOTAL_SENTENCE_LENGTH,TOTAL_JAIL_CREDITS_(IN_DAYS),NO_RESTITUTION_FLAG,P&P_COMMITMENT_STATUS_FLAG,P&P_COMMITMENT_STATUS_DATE,TOTAL_LENGTH_OF_SUPERVISION,PED_PRIOR_TO_1995_CONVERSION,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,NEW_PERIOD_OF_SUPERVISION_FLAG,TYPE_OF_OLD_PE_DATE_CODE
3525829,T683500,BA,JACKSON,CHRISTOPHER,NaN,NaN,2019-04-17,0001-01-01,ACTIVE,2019-04-04,2019-04-04,Y,STAT RAPE CHILD<=15DEF>6YRS,CUMBERLAND,91000.0,1058.0,NaN,NORMAL NORM,0001-01-01,0.0,0001-01-01,2019-04-18,08:21:38,N,NaN
3525830,T683509,BA,BYRD,RHONDA,LEA,NaN,2019-04-17,0001-01-01,ACTIVE,2019-04-02,2019-04-02,Y,EMBEZZLEMENT,FORSYTH,100000.0,0.0,NaN,NORMAL NORM,0001-01-01,0.0,0001-01-01,2019-04-18,11:52:35,N,NaN
3525831,T683918,BA,ALVARANGA,JAHMARI,RICKY,JR,2019-04-17,0001-01-01,ACTIVE,2019-03-18,2019-03-18,Y,ROBBERY W/DANGEROUS WEAPON,GUILFORD,41000.0,2.0,NaN,NORMAL NORM,0001-01-01,0.0,0001-01-01,2019-04-17,15:47:49,N,NaN
3525832,T684123,BA,SANDER,JONATHAN,NaN,NaN,2019-04-15,0001-01-01,ACTIVE,2019-04-15,2019-04-15,Y,MURDER FIRST DEGREE,WAKE,9999998.0,1116.0,NaN,NORMAL NORM,0001-01-01,0.0,0001-01-01,2019-04-16,11:37:55,N,NaN
3525833,T684187,BA,BAKER,AARON,DAWSON,NaN,2019-04-17,0001-01-01,ACTIVE,2019-04-15,2019-04-15,Y,ROBBERY W/DANGEROUS WEAPON,PITT,60200.0,309.0,NaN,NORMAL NORM,0001-01-01,0.0,0001-01-01,2019-04-17,16:10:48,N,NaN


In [4]:
court_commit_cols = ['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX', 'OFFENDER_ADMISSION/INTAKE_DATE', 
                     'NEW_PERIOD_OF_INCARCERATION_FL', 'P&P_COMMITMENT_STATUS_FLAG' , 'NEW_PERIOD_OF_SUPERVISION_FLAG']

In [5]:
court_commit['OFFENDER_ADMISSION/INTAKE_DATE'] = pd.to_datetime(court_commit['OFFENDER_ADMISSION/INTAKE_DATE'],
                                                               errors='coerce')

In [6]:
court_commit = court_commit[court_commit_cols]

In [124]:
# court_commit.to_pickle('court_commit.pkl')

In [5]:
court_commit = pd.read_pickle('data/preprocessed/court_commit.pkl')

### Inmates data
 - At the person level. All the data in the inmates table is aggregated over time, and therefore it has the risk of data leakage. Eg: Total_Sentence_Count could tell the algorithm that there will be upcoming sentences. Therefore, we want only the columns that could not change over time.

In [89]:
inmates = pd.read_csv('data/preprocessed/INMT4AA1.csv')

/Users/lorenh/Documents/DSaPP/.aequitas/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,25,26,53,62,64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [90]:
inmates.tail(5)

,INMATE_DOC_NUMBER,INMATE_LAST_NAME,INMATE_FIRST_NAME,INMATE_MIDDLE_INITIAL,INMATE_NAME_SUFFIX,INMATE_NAME_SOUNDEX_CODE,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,INMATE_ETHNIC_AFFILIATION,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN._STATUS_CODE,CUSTODY_CLASS_CODE,NEXT_CUSTODY_REVIEW_DATE,INMATE_CONTROL_STATUS_CODE,NEXT_SECURITY_REVIEW_DATE,INMATE_SPECIAL_CHARACTERISTICS,PAROLE_CASE_ANALYST,NEXT_PAROLE_COMM._REVIEW_DATE,INMATE_PRIMARY_ASSIGNMENT,INMATE_ADMISSION_DATE,ADMITTING_DIAGNOSTIC_CENTER,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,OTHER_FACILITY_CODE,CURRENT_DOP_COMMAND_CODE,CURRENT_DOP_AREA_CODE,INMATE_FACILITY_CODE,INMATE_TIME_COMP_STATUS_CODE,OLDEST_COMMIT.OF_CURRENT_INCAR,OLDEST_SNT.CMP._OF_CURR.INCAR.,OLDEST_CONVICTION_DATE,TOTAL_SENTENCE_COUNT,MOST_SERIOUS_OFFNSE_CURR_INCAR,INMATE_IS_FELON/MISDEMEANANT,CURRENT_COMMITMENT_PREFIX,CURRENT_SENTENCE_COMPONENT,TOTAL_SENTENCE_LENGTH(IN_DAYS),LENGTH_OF_CURRENT_INCARCERATN.,TERM_OF_INCARCERATION(IN_DAYS),LENGTH_OF_RULING_SENTENCES,LAST_RULING_PRD_COMMITMENT,LAST_RULING_PRD_COMPONENT,FINAL_RULING_PED,FINAL_RULING_TRD,FINAL_RULING_PRD,FINAL_RULING_MAX_RELEASE_DATE,LAW_FOR_FINAL_RULING_DATES,DAYS_SERVED_IN_DOC_CUSTODY,DATE_TRD_&_PRD_LAST_COMPUTED,ON/OFF_GAIN_TIME_CODE,GT_RATE_/_ET_LEVEL_CODE,LAST_DATE_ON/OFF_GAIN_TIME,TOTAL_DISCIPLINE_INFRACTIONS,LATEST_DISCIPLINE_INFRACTION,LAST_DISCIPLINE_INFRACTION_DT.,DATE_OF_LAST_ARREST_ON_PAROLE,CURRENT_PENDING_REVIEWS_FLAG,ESCAPE_HISTORY_FLAG,PRIOR_INCARCERATIONS_FLAG,NEXT_PAROLE_REVIEW_TYPE_CODE,TIME_OF_LAST_MOVEMENT,POPULATION/MANAGEMENT_UNIT,INMATE_POSITIVELY_IDENTIFIED,PAROLE_AND_TERMINATE_STATUS,INMATE_LABEL_STATUS_CODE,PRIMARY_OFFENSE_QUALIFIER
461266,T684058,HANSEN,JESSE,D,NaN,NaN,MALE,WHITE,1996-10-03,NaN,ACTIVE,ACTIVE,MEDIUM MED,0001-01-01,REGULAR POPULATION RPOP,0001-01-01,REGULAR,NaN,0001-01-01,NaN,2019-04-18,POLK YI DIAG CTR NEW,2019-04-18,NEW ADMISSION,NEW HANOVER COUNTY,2075,3980,POLK CI POLK,NEITHER PRD NOR PED COMPUTED,NaN,NaN,0001-01-01,0,NaN,FELON,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0001-01-01,0001-01-01,0001-01-01,0001-01-01,NaN,0,2019-04-18,NaN,NaN,0001-01-01,0,NaN,0001-01-01,0001-01-01,N,N,NaN,NaN,15:20:00,NaN,NO,NaN,NaN,NaN
461267,T684123,SANDER,JONATHAN,NaN,NaN,NaN,MALE,WHITE,1963-07-24,NaN,ACTIVE,ACTIVE,CLOSE CLS,0001-01-01,REGULAR POPULATION RPOP,0001-01-01,REGULAR,BRM11,0001-01-01,NaN,2019-04-15,CENTRAL PRISON D. C.,2019-04-15,NEW ADMISSION,WAKE COUNTY,2075,3100,CENTRAL PRISON CENT,NEITHER PRD NOR PED COMPUTED,BA,1.0,0001-01-01,0,NaN,FELON,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0001-01-01,0001-01-01,0001-01-01,0001-01-01,NaN,0,2019-04-17,NaN,NaN,0001-01-01,0,NaN,0001-01-01,0001-01-01,N,N,NaN,NaN,19:24:00,NaN,NO,NaN,NaN,NaN
461268,T684187,BAKER,AARON,D,NaN,NaN,MALE,BLACK,2001-05-28,AFRICAN,ACTIVE,ACTIVE,MEDIUM MED,0001-01-01,REGULAR POPULATION RPOP,0001-01-01,REGULAR,CKL02,0001-01-01,NaN,2019-04-17,NaN,2019-04-17,NEW ADMISSION,PITT COUNTY,2071,3720,FOOTHILLS CI FOOT,NEITHER PRD NOR PED COMPUTED,BA,1.0,0001-01-01,0,NaN,FELON,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0001-01-01,0001-01-01,0001-01-01,0001-01-01,NaN,0,2019-04-17,NaN,NaN,0001-01-01,0,NaN,0001-01-01,0001-01-01,N,N,NaN,NaN,15:09:00,NaN,NO,NaN,NaN,NaN
461269,T684216,LOFTIN,NATHAN,J,II,NaN,MALE,BLACK,1988-11-27,NaN,ACTIVE,ACTIVE,MEDIUM MED,0001-01-01,REGULAR POPULATION RPOP,0001-01-01,REGULAR,NaN,0001-01-01,NaN,2019-04-18,CENTRAL PRISON D. C.,2019-04-18,NEW ADMISSION,WAYNE COUNTY,2075,3100,CENTRAL PRISON CENT,NEITHER PRD NOR PED COMPUTED,NaN,NaN,0001-01-01,0,NaN,FELON,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0001-01-01,0001-01-01,0001-01-01,0001-01-01,NaN,0,2019-04-18,NaN,NaN,0001-01-01,0,NaN,0001-01-01,0001-01-01,N,N,NaN,NaN,15:27:00,NaN,NO,NaN,NaN,NaN
461270,T684365,RODRIGUEZ,RUBEN,G,NaN,NaN,MALE,OTHER,1988-02-09,NaN,ACTIVE,ACTIVE,MINIMUM 1 MN1,0001-01-01,REGULAR POPULATION RPOP,0001-01-01,REGULAR,NaN,0001-01-01,NaN,2019-04-18,PIEDMONT CI DIAG CTR,2019-04-18,NEW ADMISSION,ALLEGHANY COUNTY,2071,3500,PIEDMONT CI PIED,NEITHER PRD NOR PED COMPUTED,NaN,NaN,0001-01-0

In [91]:
inmates_cols = ['INMATE_DOC_NUMBER', 'INMATE_GENDER_CODE', 'INMATE_RACE_CODE', 'INMATE_BIRTH_DATE']

In [11]:
def convert_dates(date_series, date_format='%Y-%m-%d', timestamp=False):
    '''
    Faster approach to datetime parsing for large datasets leveraging repated dates.

    Attribution: https://github.com/sanand0/benchmarks/commit/0baf65b290b10016e6c5118f6c4055b0c45be2b0
    '''
    if timestamp:
        dates = {date:pd.to_datetime(date) for date in date_series.unique()}
    else:
        dates = {date: datetime.strptime(date, date_format) for date in date_series.unique()}
    return date_series.map(dates)


In [84]:
inmates['INMATE_BIRTH_DATE'].tail()

461266   1996-10-03
461267   1963-07-24
461268   2001-05-28
461269   1988-11-27
461270   1988-02-09
Name: INMATE_BIRTH_DATE, dtype: datetime64[ns]

In [93]:
inmates['INMATE_BIRTH_DATE'] =  convert_dates(inmates['INMATE_BIRTH_DATE'], '%Y-%m-%d')

In [94]:
# Dropping 218 inmates without a birthdate recorded.
inmates = inmates[inmates['INMATE_BIRTH_DATE'].notnull()]

In [95]:
inmates = inmates[inmates_cols]

In [209]:
inmates.loc[inmates['INMATE_DOC_NUMBER'] == 54932]

,INMATE_DOC_NUMBER,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE
28867,54932,MALE,WHITE,1999-09-28 00:00:00


In [123]:
# inmates.to_pickle('inmates.pkl')

In [7]:
inmates = pd.read_pickle('data/preprocessed/inmates.pkl')

### Sentence computation data 
- At the individual commitment level (person and commitment IDs), containing pertinent dates for the sentences. Sentences served consecutively for a given inmate will have the same COMMITMENT_PREFIX and subsequent SENTENCE_COMPONENTs (it seems). For our purposes, we need the initial beginning date of each sentence and the final end date.

In [13]:
sentence_computation = pd.read_csv('data/preprocessed/INMT4BB1.csv')

In [154]:
sentence_computation.tail(5)

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,INMATE_SENTENCE_COMPONENT,INMATE_COMPUTATION_STATUS_FLAG,SENTENCE_BEGIN_DATE_(FOR_MAX),ACTUAL_SENTENCE_END_DATE,PROJECTED_RELEASE_DATE_(PRD),PAROLE_DISCHARGE_DATE,PAROLE_SUPERVISION_BEGIN_DATE
1704946,1609500,BA,1,ACTIVE,2019-04-11,2019-04-17,2019-04-17,0001-01-01,0001-01-01
1704947,1609905,BA,1,ACTIVE,2019-04-17,2019-04-24,2019-04-24,0001-01-01,0001-01-01
1704948,1610013,BA,1,ACTIVE,2019-04-08,2019-04-25,2019-04-25,0001-01-01,0001-01-01
1704949,1610047,BA,1,ACTIVE,2019-04-02,2019-04-25,2019-04-25,0001-01-01,0001-01-01
1704950,1610101,BA,1,ACTIVE,2019-04-11,2019-04-25,2019-04-25,0001-01-01,0001-01-01


### Sentence data 
- At the individual commitment level. Contains data on the sentence. This will be the primary basis for our columns. For efficiency, I have ommitted some variables that could be included as dummy variables: PUNISHMENT_TYPE_CODE, COURT_TYPE_CODE, SENTENCING_PENALTY_CLASS_CODE, SENTENCING_PENALTY_CLASS_CODE, PRIOR_RECORD_LEVEL_CODE, MINIMUM_SENTENCE_LENGTH, SENTENCE_TYPE_CODE, COMPONENT_DISPOSITION_CODE. Instead I will include those most applicable to the crime itself.

In [3]:
sentences = pd.read_csv('data/preprocessed/OFNT3CE1.csv')

/mnt/data/users/lhinkson/.nctriage/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,21,25,33,41,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
sentences.loc[sentences['SENTENCE_COMPONENT_NUMBER'] > 1]

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,COUNTY_OF_CONVICTION_CODE,COURT_DOCKET_NUMBER,PUNISHMENT_TYPE_CODE,COURT_TYPE_CODE,COMPONENT_DISPOSITION_CODE,CMP._DISPOSITION_CODE_(2_OF_2),NUMBER_OF_COUNTS,TYPE_OF_COUNT_CODE,PRIMARY_OFFENSE_CODE,OFFENSE_QUALIFIER_CODE,DATE_OFFENSE_COMMITTED_-_BEGIN,DATE_OFFENSE_COMMITTED_-_END,NC_GENERAL_STATUTE_NUMBER,PRIMARY_FELONY/MISDEMEANOR_CD.,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RCD._POINTS/CONVICTIONS,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SUPERVISION_TERM_EXTENSION,SUPERVISION_TO_FOLLOW_INCAR.,SPLIT_SENTENCE_ACTIVE_TERM,G.S._MAXIMUM_SENTENCE_ALLOWED,SERVING_MIN_OR_MAX_TERM_CODE,SENTENCE_TYPE_CODE,SENTENCE_TYPE_CODE.1,SENTENCE_TYPE_CODE.2,SENTENCE_TYPE_CODE.3,SENTENCE_TYPE_CODE.4,SENTENCE_TYPE_CODE.5,CREDITS_FOR_JAIL_DAYS_SERVED,ICC_JAIL_CREDITS_(IN_DAYS),SENTENCE_CHAINING_TYPE_CODE,PRIOR_COMMITMENT_PREFIX,PRIOR_COMPONENT_IDENTIFIER,P&P_SUPV.TERM_CHAIN_TYPE_CODE,P&P_PRIOR_COMMITMENT_PREFIX,PRIOR_P&P_COMMNT/COMPONENT_ID,TIME_COMPUTATION_EXCEPTION_CD.,SENTENCE_CONVICTION_DATE,SENTENCE_EFFECTIVE(BEGIN)_DATE,DELEGATED_AUTHORITY_FLAG,INMATE_SENTENCE_STATUS_CODE,INMATE_COMPONENT_STATUS_DATE,P&P_CASE_STATUS,P&P_COMPONENT_STATUS_DATE,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,ORIGINAL_DATA_ENTRY_DATE,ORIGINAL_SENTENCE_AUDIT_CODE,DATE_OF_LAST_UPDATE_TWO,TIME_OF_LAST_UPDATE_TWO
4,4,AA,2,PERQUIMANS,83000802.0,FAIR FELONS,SUPERIOR,UNKNOWN,NaN,1,NaN,SELL SCHEDULE II,AID&ABET,0001-01-01,0001-01-01,NaN,FELON,CLASS H,0,NaN,0,30000,0.0,0.0,225.0,0000000,0.0,MAX.TERM:,DEPT OF CORR DIV OF PRISONS,NaN,NaN,NaN,NaN,NaN,8,0,CONCURRENT,AA,1.0,NaN,NaN,NaN,NOT APPLICABLE,1983-07-12,1983-07-12,NaN,ACTIVE,1983-07-13,NaN,0001-01-01,0001-01-01,01:00:00,0001-01-01,CONVERSION CC,0001-01-01,01:00:00
9,6,AB,2,CUMBERLAND,NaN,PRE-FAIR,DISTRICT,UNKNOWN,NaN,1,NaN,WORTHLESS CHECK,PRINCIPAL,0001-01-01,0001-01-01,NaN,MISD.,MISD.(PRE-STRUCTURE),0,NaN,30,0,0.0,0.0,628.0,0000000,NaN,MIN.TERM:,DEPT OF CORR DIV OF PRISONS,NaN,NaN,NaN,NaN,NaN,0,0,CONSECUTIV,AB,1.0,NaN,NaN,NaN,NOT APPLICABLE,1973-04-11,1973-04-11,NaN,ACTIVE,1973-04-15,NaN,0001-01-01,0001-01-01,01:00:00,0001-01-01,CONVERSION CS,0001-01-01,01:00:00
10,6,AB,3,CUMBERLAND,NaN,PRE-FAIR,DISTRICT,UNKNOWN,NaN,1,NaN,WORTHLESS CHECK,PRINCIPAL,0001-01-01,0001-01-01,NaN,MISD.,MISD.(PRE-STRUCTURE),0,NaN,30,0,0.0,0.0,628.0,0000000,NaN,MIN.TERM:,DEPT OF CORR DIV OF PRISONS,NaN,NaN,NaN,NaN,NaN,0,0,CONSECUTIV,AB,2.0,NaN,NaN,NaN,NOT APPLICABLE,1973-04-11,1973-04-11,NaN,ACTIVE,1973-04-15,NaN,0001-01-01,0001-01-01,01:00:00,0001-01-01,CONVERSION CS,0001-01-01,01:00:00
11,6,AB,4,CUMBERLAND,NaN,PRE-FAIR,DISTRICT,UNKNOWN,NaN,1,NaN,WORTHLESS CHECK,PRINCIPAL,0001-01-01,0001-01-01,NaN,MISD.,MISD.(PRE-STRUCTURE),0,NaN,30,0,0.0,0.0,628.0,0000000,NaN,MIN.TERM:,DEPT OF CORR DIV OF PRISONS,NaN,NaN,NaN,NaN,NaN,0,0,CONSECUTIV,AB,3.0,NaN,NaN,NaN,NOT APPLICABLE,1973-04-11,1973-04-11,NaN,ACTIVE,1973-04-15,NaN,0001-01-01,0001-01-01,01:00:00,0001-01-01,CONVERSION CS,0001-01-01,01:00:00
12,6,AB,5,CUMBERLAND,NaN,PRE-FAIR,DISTRICT,UNKNOWN,NaN,1,NaN,WORTHLESS CHECK,PRINCIPAL,0001-01-01,0001-01-01,NaN,MISD.,MISD.(PRE-STRUCTURE),0,NaN,30,0,0.0,0.0,628.0,0000000,NaN,MIN.TERM:,DEPT OF CORR DIV OF PRISONS,NaN,NaN,NaN,NaN,NaN,0,0,CONSECUTIV,AB,4.0,NaN,NaN,NaN,NOT APPLICABLE,1973-04-11,1973-04-11,NaN,ACTIVE,1973-04-15,NaN,0001-01-01,0001-01-01,01:00:00,0001-01-01,CONVERSION CS,0001-01-01,01:00:00
13,6,AB,6,CUMBERLAND,NaN,PRE-FAIR,DISTRICT,UNKNOWN,NaN,1,NaN,WORTHLESS CHECK,PRINCIPAL,0001-01-01,0001-01-01,NaN,MISD.,MISD.(PRE-STRUCTURE),0,NaN,30,0,0.0,0.0,628.0,0000000,NaN,MIN.TERM:,DEPT OF CORR DIV OF PRISONS,NaN,NaN,NaN,NaN,NaN,0,0,CONSECUTIV,AB,5.0,NaN,NaN,NaN,NOT APPLICABLE,1973-04-11,1973-04-11,NaN,ACTIVE,1973-04-15,NaN,0001-01-01,0001-01-01,01:00:00,0001-01-01,CONVERSION CS,0001-01-01,01:00:00
14,6,AB,7,CUMBERLAND,NaN,PRE-FAIR,DISTRICT,UNKNOWN,NaN,1,NaN,WORTHLESS CHECK,PRINCIPAL,0001-01-01,0001-01-01,NaN,MISD.,MISD.(PRE-STRUCTURE),0,NaN,30,

In [126]:
sentences.tail(5)

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,COUNTY_OF_CONVICTION_CODE,COURT_DOCKET_NUMBER,PUNISHMENT_TYPE_CODE,COURT_TYPE_CODE,COMPONENT_DISPOSITION_CODE,CMP._DISPOSITION_CODE_(2_OF_2),NUMBER_OF_COUNTS,TYPE_OF_COUNT_CODE,PRIMARY_OFFENSE_CODE,OFFENSE_QUALIFIER_CODE,DATE_OFFENSE_COMMITTED_-_BEGIN,DATE_OFFENSE_COMMITTED_-_END,NC_GENERAL_STATUTE_NUMBER,PRIMARY_FELONY/MISDEMEANOR_CD.,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RCD._POINTS/CONVICTIONS,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SUPERVISION_TERM_EXTENSION,SUPERVISION_TO_FOLLOW_INCAR.,SPLIT_SENTENCE_ACTIVE_TERM,G.S._MAXIMUM_SENTENCE_ALLOWED,SERVING_MIN_OR_MAX_TERM_CODE,SENTENCE_TYPE_CODE,SENTENCE_TYPE_CODE.1,SENTENCE_TYPE_CODE.2,SENTENCE_TYPE_CODE.3,SENTENCE_TYPE_CODE.4,SENTENCE_TYPE_CODE.5,CREDITS_FOR_JAIL_DAYS_SERVED,ICC_JAIL_CREDITS_(IN_DAYS),SENTENCE_CHAINING_TYPE_CODE,PRIOR_COMMITMENT_PREFIX,PRIOR_COMPONENT_IDENTIFIER,P&P_SUPV.TERM_CHAIN_TYPE_CODE,P&P_PRIOR_COMMITMENT_PREFIX,PRIOR_P&P_COMMNT/COMPONENT_ID,TIME_COMPUTATION_EXCEPTION_CD.,SENTENCE_CONVICTION_DATE,SENTENCE_EFFECTIVE(BEGIN)_DATE,DELEGATED_AUTHORITY_FLAG,INMATE_SENTENCE_STATUS_CODE,INMATE_COMPONENT_STATUS_DATE,P&P_CASE_STATUS,P&P_COMPONENT_STATUS_DATE,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,ORIGINAL_DATA_ENTRY_DATE,ORIGINAL_SENTENCE_AUDIT_CODE,DATE_OF_LAST_UPDATE_TWO,TIME_OF_LAST_UPDATE_TWO
4257487,1610101,BA,1,HENDERSON,17050540.0,ACTIVE SS,SUPERIOR,NEGOTIATED PLEA,NaN,1,CONCURRENT,AWDWISI,PRINCIPAL,2017-02-10,2017-02-10,14-33(B),FELON,CLASS E,0,LEVEL I,10800,30000,0.0,0.0,0.0,0,0.0,MAX.TERM:,DEPT OF CORR DIV OF PRISONS,JAIL POST RELEASE,NaN,NaN,NaN,NaN,791,0,INITIAL,NaN,NaN,NaN,NaN,NaN,FLAT TIME JAIL CRDT.,2019-04-11,2019-04-11,NaN,ACTIVE,2019-04-11,NORMAL NORM,2019-04-11,2019-04-18,13:47:15,2019-04-18,NaN,2019-04-18,13:46:08
4257488,1610104,01,1,PITT,18052768.0,DWI,DISTRICT,GUILTY,NaN,1,CONCURRENT,DWI LEVEL 5,PRINCIPAL,2018-04-16,2018-04-16,NaN,MISD.,NON CLASS CODE,0,NaN,0,0,1200.0,0.0,0.0,0,NaN,MAX.TERM:,PROBATION,SUSPENDED SENTENCE,DWI CONVICTION,NaN,NaN,NaN,0,0,NaN,NaN,NaN,INITIAL,NaN,NaN,NOT APPLICABLE,2019-04-18,2019-04-18,N,NaN,0001-01-01,NORMAL NORM,2019-04-18,2019-04-18,15:12:20,2019-04-18,NaN,2019-04-18,15:12:20
4257489,1610118,01,1,WAKE,18002289.0,COMMUNITY SS (DCC),DISTRICT,BENCH TRIAL,BENCH TRIAL,1,CONCURRENT,RECKLESS DRIVING,PRINCIPAL,2018-07-21,2018-07-21,20-140(B),MISD.,CLASS 2 MISDEMEANOR SS,2,LEVEL II,0,0,1200.0,0.0,0.0,0,NaN,MAX.TERM:,PROBATION,SUSPENDED SENTENCE,COUNTY JAIL,NaN,NaN,NaN,0,0,NaN,NaN,NaN,INITIAL,NaN,NaN,NOT APPLICABLE,2019-04-18,2019-04-18,Y,NaN,0001-01-01,NORMAL NORM,2019-04-18,2019-04-18,16:01:56,2019-04-18,NaN,2019-04-18,16:01:56
4257490,1610128,01,1,SAMPSON,17052701.0,DWI,DISTRICT,GUILTY,GUILTY,1,CONCURRENT,DWI LEVEL 5,PRINCIPAL,2017-10-01,2017-10-01,20-138.1,MISD.,NON CLASS CODE,0,NaN,0,0,1200.0,0.0,0.0,0,NaN,MAX.TERM:,PROBATION,SUSPENDED SENTENCE,DWI CONVICTION,NaN,NaN,NaN,0,0,NaN,NaN,NaN,INITIAL,NaN,NaN,NOT APPLICABLE,2019-04-18,2019-04-18,N,NaN,0001-01-01,NORMAL NORM,2019-04-18,2019-04-18,17:26:48,2019-04-18,NaN,2019-04-18,17:26:48
4257491,T153879,01,1,DARE,99003876.0,COMMUNITY SS (DCC),SUPERIOR,GUILTY,GUILTY,1,CONCURRENT,CONSPIRACY,PRINCIPAL,1999-07-02,1999-07-02,90 95,FELON,CLASS H,0,LEVEL I,600,800,NaN,NaN,NaN,0,NaN,MAX.TERM:,PROBATION,SUSPENDED SENTENCE,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,INITIAL,NaN,NaN,NOT APPLICABLE,1999-09-27,1999-09-27,N,NaN,0001-01-01,TOLLED TOLLD,2000-09-26,2000-10-20,11:20:45,1999-10-07,NaN,2000-09-26,09:07:51


In [127]:
sentences_cols =  ["OFFENDER_NC_DOC_ID_NUMBER", "COMMITMENT_PREFIX", "SENTENCE_COMPONENT_NUMBER", 
                   "NUMBER_OF_COUNTS", "PRIMARY_OFFENSE_CODE", # Want to group this one
                   "PRIMARY_FELONY/MISDEMEANOR_CD.", "COUNTY_OF_CONVICTION_CODE", # Want to group this one too
                   "PRIOR_RCD._POINTS/CONVICTIONS", "MINIMUM_SENTENCE_LENGTH",
                   "MAXIMUM_SENTENCE_LENGTH", "LENGTH_OF_SUPERVISION", "SERVING_MIN_OR_MAX_TERM_CODE",
                   "SENTENCE_EFFECTIVE(BEGIN)_DATE"
                  ]

In [128]:
# sentences["SENTENCE_EFFECTIVE(BEGIN)_DATE"] = pd.to_datetime(sentences["SENTENCE_EFFECTIVE(BEGIN)_DATE"],
#                                                              errors = "coerce")

sentences["SENTENCE_EFFECTIVE(BEGIN)_DATE"] =  convert_dates(sentences["SENTENCE_EFFECTIVE(BEGIN)_DATE"], '%Y-%m-%d')

In [129]:
sentences = sentences[sentences_cols]

In [8]:
# sentences.to_pickle('sentences.pkl')
sentences = pd.read_pickle('data/preprocessed/sentences.pkl')

### Disciplinary infraction data 
- At the ID number and date level. These must be joined and filtered by date, not commitment_prefix. 

In [103]:
discipline = pd.read_csv('data/preprocessed/INMT9CF1.csv')

/Users/lorenh/Documents/DSaPP/.aequitas/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (8,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [104]:
discipline.tail()

,INMATE_DOC_NUMBER,DISCIPLINARY_INFRACTION_DATE,DISCIPLINARY_INFRACTION_TIME,DISCIP._INFRACTION_SEQUENCE_#,DISCIPLINARY_INFRACTION_CODE,DISCIPLINARY_CHARGE_LEVEL,INMATE_PLEA_RE._INFRACTION,DISCI.INFRACTION_VERDICT_CODE,TYPE_OF_HEARING_FOR_PUNISHMENT,DISCIPLINARY_APPEAL_DECISION,DISCI._SEGREGATION_TIME_(DAYS),DSEG_DAYS_SUSPENDED(IN_MONTHS),GOOD_TIME_LOST_DUE_TO_INFRAC.,GOOD_TIME_LOST_SUSPENDED_(MOS),DISCI._VIOLATION_STATUS_CODE,EXTRA_DUTY_HOURS,EXTRA_DUTY_HOURS_SUSPEND_MNTHS,PRIVILEGE_LOST_-_1ST_TYPE_CODE,PRIVILEGE_LOST_-_2ND_TYPE_CODE,PRIVILEGE_LOST_-_3RD_TYPE_CODE,PRIVILEGES_SUSPENDED_(IN_DAYS),SUSP.PRIVILEGES_SUSPENDED(MOS),CUSTODY_DEMOTED_FROM_CODE,CUSTODY_DEMOTED_TO_CODE,CUSTODY_DEMOTION_SUSPENDED(MO),ACTIVATE_PRIOR_SUSPENSION,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,ORIGINAL_DR_CODE_FROM_INVEST.,SUSPENSION_STATUS,DATE_OF_PRIOR_SUSP_PUNISH,TIME_OF_PRIOR_SUSP_PUNISH
3316819,1608099,2019-04-12,17:38:00,1,DISOBEY ORDER,UNIT,GUILTY,GUILTY,NaN,NaN,5.0,NaN,1.0,NaN,APPLIED,30.0,NaN,RADIO,CANTEEN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,2019-04-18,13:09:15,DISOBEY ORDER,NaN,0001-01-01,00:00:01
3316820,1608374,2019-04-10,22:45:00,1,HIGH RISK ACT,UNIT,NO PLEA,COUNSELED,NaN,NaN,NaN,NaN,NaN,NaN,APPLIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-10,23:24:45,HIGH RISK ACT,NaN,0001-01-01,00:00:01
3316821,1608387,2019-04-04,10:40:00,1,DISOBEY ORDER,UNIT,NO PLEA,DISMISSED,NaN,NaN,NaN,NaN,NaN,NaN,APPLIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-05,14:20:42,DISOBEY ORDER,NaN,0001-01-01,00:00:01
3316822,1608438,2019-04-12,11:40:00,1,DISOBEY ORDER,UNIT,NOT GUILTY,REFER DHO,NaN,NaN,NaN,NaN,NaN,NaN,APPLIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-18,09:16:06,DISOBEY ORDER,NaN,0001-01-01,00:00:01
3316823,1609519,2019-04-15,10:25:00,1,SUBSTANCE POSSESSION,UNIT,NO PLEA,REFER DHO,NaN,NaN,NaN,NaN,NaN,NaN,APPLIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-17,20:15:10,SUBSTANCE POSSESSION,NaN,0001-01-01,00:00:01


In [105]:
discipline_cols = ['INMATE_DOC_NUMBER', 'DISCIPLINARY_INFRACTION_DATE', 
                   'DISCIPLINARY_INFRACTION_CODE', 'DISCI._SEGREGATION_TIME_(DAYS)']

In [106]:
# discipline['DISCIPLINARY_INFRACTION_DATE'] = pd.to_datetime(discipline['DISCIPLINARY_INFRACTION_DATE'], errors='coerce')

discipline["DISCIPLINARY_INFRACTION_DATE"] =  convert_dates(discipline["DISCIPLINARY_INFRACTION_DATE"], '%Y-%m-%d')


In [107]:
discipline = discipline[discipline_cols]

In [9]:
# discipline.to_pickle('discipline.pkl')
discipline = pd.read_pickle('data/preprocessed/discipline.pkl')

# Creating the Dataset

First, I will use sentence_computation to conglomerate all of the different components of a single commitment to get information at the continous internment level.

### Get the Dates of a Single Internment

In [14]:
# sentence_computation["SENTENCE_BEGIN_DATE_(FOR_MAX)"] = pd.to_datetime(sentence_computation["SENTENCE_BEGIN_DATE_(FOR_MAX)"], errors = "coerce")
# sentence_computation["ACTUAL_SENTENCE_END_DATE"] = pd.to_datetime(sentence_computation["ACTUAL_SENTENCE_END_DATE"], errors = "coerce")
# sentence_computation["PROJECTED_RELEASE_DATE_(PRD)"] = pd.to_datetime(sentence_computation["PROJECTED_RELEASE_DATE_(PRD)"], errors = "coerce")

sentence_computation["SENTENCE_BEGIN_DATE_(FOR_MAX)"] = convert_dates(sentence_computation["SENTENCE_BEGIN_DATE_(FOR_MAX)"])
sentence_computation["ACTUAL_SENTENCE_END_DATE"] = convert_dates(sentence_computation["ACTUAL_SENTENCE_END_DATE"])
sentence_computation["PROJECTED_RELEASE_DATE_(PRD)"] = convert_dates(sentence_computation["PROJECTED_RELEASE_DATE_(PRD)"])


In [15]:
# sentence_computation["PAROLE_SUPERVISION_BEGIN_DATE"] = pd.to_datetime(sentence_computation["PAROLE_SUPERVISION_BEGIN_DATE"], errors = "coerce")
# sentence_computation["PAROLE_DISCHARGE_DATE"] = pd.to_datetime(sentence_computation["PAROLE_DISCHARGE_DATE"], errors = "coerce")

sentence_computation["PAROLE_SUPERVISION_BEGIN_DATE"] = convert_dates(sentence_computation["PAROLE_SUPERVISION_BEGIN_DATE"])
sentence_computation["PAROLE_DISCHARGE_DATE"] = convert_dates(sentence_computation["PAROLE_DISCHARGE_DATE"])


In [19]:
sentence_computation['PROJECTED_RELEASE_DATE_(PRD)'].describe()

count                 1704951
unique                  27040
top       0001-01-01 00:00:00
freq                    19536
Name: PROJECTED_RELEASE_DATE_(PRD), dtype: object

In [20]:
sentence_computation['PROJECTED_RELEASE_DATE_(PRD)'].max()

datetime.datetime(9999, 1, 3, 0, 0)

In [23]:
import seaborn as sns

In [30]:
date_diff = lambda x, y: (x - y).days if (0 < x.year < 2622) and (0< y.year < 2622) else pd.np.nan


#  lambda x, y: (x - y).days if (x != datetime.strptime('0001-01-01', '%Y-%m-%d')) and (y != datetime.strptime('0001-01-01', '%Y-%m-%d')) else pd.np.nan

In [ ]:
projected_vs_actual = sentence_computation[["ACTUAL_SENTENCE_END_DATE","PROJECTED_RELEASE_DATE_(PRD)"]].apply(lambda x:date_diff(*x), axis=1)



In [ ]:
projected_vs_actual.hist()

In [158]:
sentence_computation['SENTENCE_END'] = sentence_computation["ACTUAL_SENTENCE_END_DATE"]\
                                         .fillna(sentence_computation["PROJECTED_RELEASE_DATE_(PRD)"])



In [137]:
# sentence_computation.to_pickle('sentence_computation.pkl')
sentence_computation = pd.read_pickle('sentence_computation.pkl')

In [138]:
# For each commitment, getting the lowest beginning date and the highest end date for the full sentence term.
sentence_subset = sentence_computation.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX'])\
        .agg({'SENTENCE_BEGIN_DATE_(FOR_MAX)': min, 
              'SENTENCE_END': max,
              'PAROLE_SUPERVISION_BEGIN_DATE': min,
              'PAROLE_DISCHARGE_DATE': max
             }
            ).reset_index()

In [149]:
# Null end dates encode life sentence_computations, so I will set the sentence_end to 2230-1-1 (near top of pandas date range)
# sentence_subset.loc[sentence_subset.SENTENCE_END.isnull(), "SENTENCE_END"] = pd.to_datetime('2230-1-1')

sentence_subset.loc[sentence_subset.SENTENCE_END.isnull(), "SENTENCE_END"] = datetime.strptime('2230-01-01', '%Y-%m-%d')

In [140]:
sentence_subset.tail()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_SUPERVISION_BEGIN_DATE,PAROLE_DISCHARGE_DATE
885123,1609500,BA,2019-04-11 00:00:00,2019-04-17 00:00:00,0001-01-01 00:00:00,0001-01-01 00:00:00
885124,1609905,BA,2019-04-17 00:00:00,2019-04-24 00:00:00,0001-01-01 00:00:00,0001-01-01 00:00:00
885125,1610013,BA,2019-04-08 00:00:00,2019-04-25 00:00:00,0001-01-01 00:00:00,0001-01-01 00:00:00
885126,1610047,BA,2019-04-02 00:00:00,2019-04-25 00:00:00,0001-01-01 00:00:00,0001-01-01 00:00:00
885127,1610101,BA,2019-04-11 00:00:00,2019-04-25 00:00:00,0001-01-01 00:00:00,0001-01-01 00:00:00


In [150]:
sentence_subset.loc[sentence_subset['PAROLE_DISCHARGE_DATE'] != datetime.strptime('0001-01-01', '%Y-%m-%d')]

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_SUPERVISION_BEGIN_DATE,PAROLE_DISCHARGE_DATE
16,28,BA,1994-06-15 00:00:00,1997-08-22 00:00:00,0001-01-01 00:00:00,1998-12-28 00:00:00
19,32,BA,1992-08-05 00:00:00,1994-09-12 00:00:00,0001-01-01 00:00:00,1999-09-01 00:00:00
27,35,BA,1991-09-16 00:00:00,1997-05-20 00:00:00,0001-01-01 00:00:00,1997-08-18 00:00:00
29,35,BC,2001-01-16 00:00:00,2011-11-29 00:00:00,0001-01-01 00:00:00,2012-08-25 00:00:00
43,52,BA,2015-12-03 00:00:00,2016-09-27 00:00:00,0001-01-01 00:00:00,2017-06-24 00:00:00
78,106,BA,1995-04-13 00:00:00,1995-08-17 00:00:00,1995-08-17 00:00:00,1996-06-16 00:00:00
92,133,BA,1992-09-08 00:00:00,2009-09-08 00:00:00,0001-01-01 00:00:00,2009-09-08 00:00:00
100,142,BB,2017-07-14 00:00:00,2018-08-11 00:00:00,0001-01-01 00:00:00,2018-09-14 00:00:00
108,147,BA,1994-10-17 00:00:00,1996-11-06 00:00:00,0001-01-01 00:00:00,1996-11-06 00:00:00
114,156,BA,1995-03-15 00:00:00,1996-05-14 00:00:00,1996-05-14 00:00:00,1996-07-22 00:00:00


In [165]:
parole_calc = lambda x, y: (x - y).days if (x != datetime.strptime('0001-01-01', '%Y-%m-%d')) and (y != datetime.strptime('0001-01-01', '%Y-%m-%d')) else pd.np.nan

parole_calc(sentence_subset.loc[92,'PAROLE_DISCHARGE_DATE'], sentence_subset.loc[92,'PAROLE_SUPERVISION_BEGIN_DATE'])

nan

In [166]:
# Get the number of parole days, then drop the parole date columns
# sentence_subset['PAROLE_DAYS'] = (sentence_subset['PAROLE_DISCHARGE_DATE'] - 
#                                   sentence_subset['PAROLE_SUPERVISION_BEGIN_DATE']) / np.timedelta64(1, 'D')

sentence_subset.loc[:,'PAROLE_DAYS'] = sentence_subset[
    ['PAROLE_DISCHARGE_DATE','PAROLE_SUPERVISION_BEGIN_DATE']].apply(lambda x: parole_calc(*x), axis=1)

In [195]:
sentence_subset['PAROLE_DAYS'].dtype

dtype('float64')

In [173]:
sentence_subset.loc[sentence_subset['PAROLE_DISCHARGE_DATE'] != datetime.strptime('0001-01-01', '%Y-%m-%d')].head(20)

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_SUPERVISION_BEGIN_DATE,PAROLE_DISCHARGE_DATE,PAROLE_DAYS
16,28,BA,1994-06-15 00:00:00,1997-08-22 00:00:00,0001-01-01 00:00:00,1998-12-28 00:00:00,0.0
19,32,BA,1992-08-05 00:00:00,1994-09-12 00:00:00,0001-01-01 00:00:00,1999-09-01 00:00:00,0.0
27,35,BA,1991-09-16 00:00:00,1997-05-20 00:00:00,0001-01-01 00:00:00,1997-08-18 00:00:00,0.0
29,35,BC,2001-01-16 00:00:00,2011-11-29 00:00:00,0001-01-01 00:00:00,2012-08-25 00:00:00,0.0
43,52,BA,2015-12-03 00:00:00,2016-09-27 00:00:00,0001-01-01 00:00:00,2017-06-24 00:00:00,0.0
78,106,BA,1995-04-13 00:00:00,1995-08-17 00:00:00,1995-08-17 00:00:00,1996-06-16 00:00:00,304.0
92,133,BA,1992-09-08 00:00:00,2009-09-08 00:00:00,0001-01-01 00:00:00,2009-09-08 00:00:00,0.0
100,142,BB,2017-07-14 00:00:00,2018-08-11 00:00:00,0001-01-01 00:00:00,2018-09-14 00:00:00,0.0
108,147,BA,1994-10-17 00:00:00,1996-11-06 00:00:00,0001-01-01 00:00:00,1996-11-06 00:00:00,0.0
114,156,BA,1995-03-15 00:00:00,1996-05-14 00:00:00,1996-05-14 00:00:00,1996-07-22 00:00:00,69.0


In [169]:
sentence_subset.fillna({'PAROLE_DAYS':0}, inplace=True)


In [172]:
sentence_subset.loc[sentence_subset['PAROLE_DISCHARGE_DATE'] != datetime.strptime('0001-01-01', '%Y-%m-%d')].head(20)

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_SUPERVISION_BEGIN_DATE,PAROLE_DISCHARGE_DATE,PAROLE_DAYS
16,28,BA,1994-06-15 00:00:00,1997-08-22 00:00:00,0001-01-01 00:00:00,1998-12-28 00:00:00,0.0
19,32,BA,1992-08-05 00:00:00,1994-09-12 00:00:00,0001-01-01 00:00:00,1999-09-01 00:00:00,0.0
27,35,BA,1991-09-16 00:00:00,1997-05-20 00:00:00,0001-01-01 00:00:00,1997-08-18 00:00:00,0.0
29,35,BC,2001-01-16 00:00:00,2011-11-29 00:00:00,0001-01-01 00:00:00,2012-08-25 00:00:00,0.0
43,52,BA,2015-12-03 00:00:00,2016-09-27 00:00:00,0001-01-01 00:00:00,2017-06-24 00:00:00,0.0
78,106,BA,1995-04-13 00:00:00,1995-08-17 00:00:00,1995-08-17 00:00:00,1996-06-16 00:00:00,304.0
92,133,BA,1992-09-08 00:00:00,2009-09-08 00:00:00,0001-01-01 00:00:00,2009-09-08 00:00:00,0.0
100,142,BB,2017-07-14 00:00:00,2018-08-11 00:00:00,0001-01-01 00:00:00,2018-09-14 00:00:00,0.0
108,147,BA,1994-10-17 00:00:00,1996-11-06 00:00:00,0001-01-01 00:00:00,1996-11-06 00:00:00,0.0
114,156,BA,1995-03-15 00:00:00,1996-05-14 00:00:00,1996-05-14 00:00:00,1996-07-22 00:00:00,69.0


In [174]:
sentence_subset.drop(['PAROLE_SUPERVISION_BEGIN_DATE', 'PAROLE_DISCHARGE_DATE'], axis=1, inplace=True, errors='ignore')


In [175]:
sentence_subset.shape

(885128, 5)

In [176]:
sentence_subset.to_pickle('sentence_subset.pkl')
# sentence_subset = pd.read_pickle('sentence_subset.pkl')

In [226]:
sentence_subset = pd.read_pickle('data/preprocessed/sentence_subset.pkl')

### Check for coding of life sentences as year 9999

In [207]:
sentence_subset.loc[sentence_subset['SENTENCE_END'] >= datetime.strptime('9998-01-01', '%Y-%m-%d')]

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS
68,80,BA,1985-11-04 00:00:00,9999-01-02 00:00:00,0.0
121,167,BA,1996-08-14 00:00:00,9999-01-02 00:00:00,0.0
241,289,BA,1982-11-08 00:00:00,9999-01-02 00:00:00,0.0
298,353,BA,1991-02-14 00:00:00,9999-01-02 00:00:00,0.0
345,397,BA,1994-05-09 00:00:00,9999-01-02 00:00:00,0.0
354,400,BA,1984-05-22 00:00:00,9999-01-02 00:00:00,0.0
807,759,BB,2013-10-28 00:00:00,9999-01-02 00:00:00,0.0
1146,1083,BA,1990-05-25 00:00:00,9999-01-02 00:00:00,0.0
1490,1364,BA,1987-08-12 00:00:00,9999-01-02 00:00:00,0.0
1547,1414,BA,1985-05-30 00:00:00,9999-01-02 00:00:00,0.0


### Bringing In the Court Commitments Table

Now I will join the sentencing dates derived above to the courts data. The resulting table is called `admissions`.

In [177]:
admissions = sentence_subset.merge(court_commit, left_on = ["INMATE_DOC_NUMBER", "INMATE_COMMITMENT_PREFIX"],
                                     right_on=['OFFENDER_NC_DOC_ID_NUMBER','COMMITMENT_PREFIX'])
admissions.drop(['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX'], axis=1, inplace=True)

In [178]:
admissions.tail(3)

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG
882792,1577605,BA,2018-04-17 00:00:00,2028-11-18 00:00:00,0.0,2018-04-28,Y,NORMAL NORM,N
882793,1577614,BA,2018-04-16 00:00:00,2019-10-26 00:00:00,0.0,2018-04-30,Y,NORMAL NORM,N
882794,1577620,BA,2018-12-17 00:00:00,2019-03-17 00:00:00,0.0,2019-01-16,Y,NORMAL NORM,N


In [179]:
admissions.shape

(882795, 9)

In [210]:
admissions.loc[admissions['INMATE_DOC_NUMBER'] == 54932]

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE
62252,54932,AA,1973-05-21 00:00:00,1973-09-26 00:00:00,0.0,1973-05-22,Y,NaN,NaN,MALE,WHITE,1999-09-28 00:00:00,-26.0


### Bringing In the Inmates Table

Now I will add the information from the inmates table. This contains demographics and date of birth. I will use this to calculate age at release.

In [180]:
admissions = admissions.merge(inmates, on='INMATE_DOC_NUMBER')

In [181]:
admissions['SENTENCE_END'].dtype

dtype('O')

In [182]:
admissions.to_pickle('admissions.pkl')

In [ ]:
# admissions = pd.read_pickle('admissions.pkl')

In [200]:
release_age_calc = lambda x, y: relativedelta(x, y).years if (x != datetime.strptime('0001-01-01', '%Y-%m-%d')) and (y != datetime.strptime('0001-01-01', '%Y-%m-%d')) else pd.np.nan
# test lambda function
release_age_calc(admissions.loc[0,'SENTENCE_END'],admissions.loc[0,'INMATE_BIRTH_DATE'])

22

In [190]:
# make sure no birthdates are missing
admissions.loc[admissions['INMATE_BIRTH_DATE'].isna()]

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE


In [191]:
# make sure no sentence end dates are missing
admissions.loc[admissions['SENTENCE_END'].isna()]

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE


In [201]:
# admissions['AGE_AT_RELEASE'] = ((admissions['SENTENCE_END'] - admissions['INMATE_BIRTH_DATE']) 
#                                 / np.timedelta64(1, 'Y')).astype(int)

admissions.loc[:,'AGE_AT_RELEASE'] = admissions[
    ['SENTENCE_END','INMATE_BIRTH_DATE']].apply(lambda x: release_age_calc(*x), axis=1)


In [203]:
admissions.loc[admissions['INMATE_DOC_NUMBER'].isin([1523385, 1523389, 1523408])]

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE
870541,1523385,BA,2016-10-05 00:00:00,2018-05-10 00:00:00,365.0,2016-11-04,Y,ENT CORR INST CORR,Y,MALE,OTHER,1989-07-11 00:00:00,28.0
870542,1523389,BA,2016-11-07 00:00:00,2019-02-28 00:00:00,1825.0,2016-12-01,Y,NORMAL NORM,N,MALE,WHITE,1984-06-15 00:00:00,34.0
870544,1523408,BA,2016-11-07 00:00:00,2026-08-18 00:00:00,0.0,2016-11-17,Y,NORMAL NORM,N,MALE,WHITE,1976-08-19 00:00:00,49.0


In [218]:
admissions.loc[(admissions['AGE_AT_RELEASE'] < 18) & (admissions['AGE_AT_RELEASE'] > 0)].shape

(13475, 13)

In [219]:
admissions.loc[(admissions['AGE_AT_RELEASE'] < 15) & (admissions['AGE_AT_RELEASE'] > 0)].shape

(42, 13)

In [220]:
admissions.loc[(admissions['AGE_AT_RELEASE'] <= 0)].shape

(31, 13)

In [226]:
admissions.loc[admissions['AGE_AT_RELEASE'].isna()].shape

(0, 13)

In [222]:
admissions_with_negatives = admissions.loc[:,:]

In [223]:
admissions = admissions.loc[(admissions['AGE_AT_RELEASE'] > 0)]

In [228]:
admissions.loc[:,'AGE_AT_RELEASE'] = admissions.loc[:,'AGE_AT_RELEASE'].astype('int')

In [ ]:
# admissions.drop('INMATE_BIRTH_DATE', axis=1, inplace=True)

In [229]:
admissions.tail(3)

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE
874459,1566234,BA,2019-01-14 00:00:00,2019-06-26 00:00:00,0.0,2019-02-05,Y,NORMAL NORM,N,MALE,BLACK,1996-12-20 00:00:00,22
874460,1566265,BA,2018-07-23 00:00:00,2018-10-01 00:00:00,270.0,2018-08-10,Y,NORMAL NORM,N,FEMALE,WHITE,1968-08-03 00:00:00,50
874461,1566292,BA,2018-01-02 00:00:00,2018-03-28 00:00:00,365.0,2018-01-12,Y,NORMAL NORM,Y,MALE,BLACK,1996-04-02 00:00:00,21


In [230]:
admissions.loc[admissions['INMATE_DOC_NUMBER'].isin([1523385, 1523389, 1523408])]

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE
870541,1523385,BA,2016-10-05 00:00:00,2018-05-10 00:00:00,365.0,2016-11-04,Y,ENT CORR INST CORR,Y,MALE,OTHER,1989-07-11 00:00:00,28
870542,1523389,BA,2016-11-07 00:00:00,2019-02-28 00:00:00,1825.0,2016-12-01,Y,NORMAL NORM,N,MALE,WHITE,1984-06-15 00:00:00,34
870544,1523408,BA,2016-11-07 00:00:00,2026-08-18 00:00:00,0.0,2016-11-17,Y,NORMAL NORM,N,MALE,WHITE,1976-08-19 00:00:00,49


In [231]:
admissions.shape

(864174, 13)

In [7]:
# Saving an intermediate table for easy loading
# admissions.to_pickle('admissions.pkl')

In [8]:
# Load the above pickle
admissions = pd.read_pickle('data/preprocessed/admissions.pkl')

### Bringing In the Sentences Table

Now I will add the information from the sentences table. This contains the information most pertinent: the crime, punishment, etc. This new table will be called `sentence_info`. Recall that the `sentences` table contains all information for every component of each sentences.

In [155]:
sentence_info = sentences.merge(admissions, left_on=['OFFENDER_NC_DOC_ID_NUMBER','COMMITMENT_PREFIX'],
                                            right_on=["INMATE_DOC_NUMBER", "INMATE_COMMITMENT_PREFIX"])

sentence_info.drop(['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX'], axis=1, inplace=True)

In [156]:
sentence_info.columns

Index(['SENTENCE_COMPONENT_NUMBER', 'NUMBER_OF_COUNTS', 'PRIMARY_OFFENSE_CODE',
       'PRIMARY_FELONY/MISDEMEANOR_CD.', 'COUNTY_OF_CONVICTION_CODE',
       'PRIOR_RCD._POINTS/CONVICTIONS', 'MINIMUM_SENTENCE_LENGTH',
       'MAXIMUM_SENTENCE_LENGTH', 'LENGTH_OF_SUPERVISION',
       'SERVING_MIN_OR_MAX_TERM_CODE', 'SENTENCE_EFFECTIVE(BEGIN)_DATE',
       'INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX',
       'SENTENCE_BEGIN_DATE_(FOR_MAX)', 'SENTENCE_END', 'PAROLE_DAYS',
       'OFFENDER_ADMISSION/INTAKE_DATE', 'NEW_PERIOD_OF_INCARCERATION_FL',
       'P&P_COMMITMENT_STATUS_FLAG', 'NEW_PERIOD_OF_SUPERVISION_FLAG',
       'INMATE_GENDER_CODE', 'INMATE_RACE_CODE', 'INMATE_BIRTH_DATE',
       'AGE_AT_RELEASE'],
      dtype='object')

### Skip this Section for Triage! 
We want to turn the categorical variables into counts/percentages.

For PRIMARY_OFFENSE_CODE and COUNTY_OF_CONVICTION_CODE, I will reduce the number of categories by grouping together anything representing less than 0.1% and 0.5% of the offenses and counties respectively into "OTHER". This gives us 125 offense groups and 58 county groups.

In [67]:
# # Offense grouping
# offense_percentages = sentence_info["PRIMARY_OFFENSE_CODE"].value_counts(normalize=True, dropna=False)
# offense_groupings = {i:i for i in sentence_info["PRIMARY_OFFENSE_CODE"].unique()}

# for i, j in offense_percentages[offense_percentages < 0.001].iteritems():
#     offense_groupings[i] = "OTHER"

In [68]:
# # County grouping
# county_percentages = sentence_info["COUNTY_OF_CONVICTION_CODE"].value_counts(normalize=True, dropna=False)
# county_groupings = {i:i for i in sentence_info["COUNTY_OF_CONVICTION_CODE"].unique()}

# for i, j in county_percentages[county_percentages < 0.005].iteritems():
#     county_groupings[i] = "OTHER"

In [85]:
# sentence_info["PRIMARY_OFFENSE_CODE"] = [offense_groupings[j] for i,j in sentence_info['PRIMARY_OFFENSE_CODE'].iteritems()]

In [238]:
# sentence_info["COUNTY_OF_CONVICTION_CODE"] = [county_groupings[j] for i,j in sentence_info['COUNTY_OF_CONVICTION_CODE'].iteritems()]

Getting count of rows for each commitment, to normalize some of the variables below. 

In [107]:
# num_distinct_rows = sentence_info.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX'])['NUMBER_OF_COUNTS'].count()
num_distinct_rows = sentence_info.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX','SENTENCE_EFFECTIVE(BEGIN)_DATE'])['NUMBER_OF_COUNTS'].count()

In [109]:
num_distinct_rows.head(15)

INMATE_DOC_NUMBER  INMATE_COMMITMENT_PREFIX  SENTENCE_EFFECTIVE(BEGIN)_DATE
6                  AA                        1973-01-30 00:00:00                1
                   AB                        1973-04-11 00:00:00               27
8                  AA                        1990-04-09 00:00:00                1
                   AB                        1993-08-30 00:00:00                1
10                 AB                        1975-05-30 00:00:00                2
14                 AA                        1975-08-18 00:00:00                2
                   AB                        1977-06-17 00:00:00                1
19                 AA                        1988-05-05 00:00:00                1
                                             1988-06-15 00:00:00                1
                                             1988-07-14 00:00:00                1
                                             1988-09-20 00:00:00                1
23                 AA 

Getting total number of counts per commitment, percentages by county groups defined above, percentages by offense groups defined above, percentages by misdemeanor/felony, percentages by the flag 'serving min or max sentence', and the means of the minimum sentence lengths, maximum sentence lengths, and length of supervision.

In [177]:
# counts = sentence_info.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX'])['NUMBER_OF_COUNTS'].sum().to_frame()
counts = sentence_info.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 'SENTENCE_EFFECTIVE(BEGIN)_DATE'])['NUMBER_OF_COUNTS'].sum().to_frame()
# counts.shape
counts

NUMBER_OF_COUNTS
INMATE_DOC_NUMBER INMATE_COMMITMENT_PREFIX SENTENCE_EFFECTIVE(BEGIN)_DATE                  
4                 AA                       1983-07-12                                     2
6                 AA                       1973-01-30                                     1
                  AB                       1973-04-11                                    27
8                 AA                       1990-04-09                                     1
                  AB                       1993-08-30                                     1
10                AA                       1975-06-11                                     1
                  AB                       1975-05-30                                     2
                                           1975-06-11                                     1
14                AA                       1975-08-18                                     2
                  AB                       1977-06-17                                     1
17                AA                       1992-08-18                                     1
19                AA                       1988-05-05                                     1
                                           1988-06-15                                     2
                                           1988-07-14                                     1
                                           1988-09-20                                     2
23                AA                       1977-07-19                                     1
26                AA                       1983-07-18                                     1
28                AA                       1991-02-15                                     1
                  BA                       1994-06-15                                     3
                  BB                       2000-10-09                                     1
31                AA                       1971-08-13                                     1
                                           1972-08-08                                     1
32                BA                       1992-08-05                                     1
                                           1992-08-25                                    15
                                           1992-09-02                                     1
                                           1992-09-09                                     6
                                           1993-01-13                                     1
                                           1993-02-24                                     1
33                AA                       1978-03-23                                     1
                  AB                       1979-11-06                                     1
                  AC                       1981-12-14                                     1
                  AD                       1984-02-02                                     1
                                           1989-02-14                                     1
                  BA                       1995-06-22                                     3
                                           1995-06-23                                     2
34                AA                       1988-09-21                                     1
                  AB                       1990-09-24                                     1
35                BA                       1991-09-16                                     1
                                           1992-01-08                                     1
                  BB                       1999-03-03                                     3
                  BC                       2001-01-16                                     1
                                           2004-10-08                                     1
37                BA       

In [115]:
counts.columns = ['COUNTS_AT_SENTENCE_EFFECTIVE_DATE']

In [241]:
# counties = sentence_info.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 
#                                   'COUNTY_OF_CONVICTION_CODE']).size().unstack(fill_value=0)

# counties.columns = ["COUNTY_" + str(i) for i in counties.columns]
# for col in counties.columns:
#     counties[col] = counties[col]/num_distinct_rows
    
# counties.shape

(854066, 58)

In [242]:
# offenses = sentence_info.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 
#                                   'PRIMARY_OFFENSE_CODE']).size().unstack(fill_value=0)

# offenses.columns = ["OFFENSE_" + str(i) for i in offenses.columns]
# for col in offenses.columns:
#     offenses[col] = offenses[col]/num_distinct_rows
    
# offenses.shape

(854066, 124)

In [243]:
# misd_felon = sentence_info.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 
#                                     'PRIMARY_FELONY/MISDEMEANOR_CD.']).size().unstack(fill_value=0)

# for col in misd_felon.columns:
#     misd_felon[col] = misd_felon[col]/num_distinct_rows
    
# misd_felon.shape

(854066, 2)

In [105]:
# min_max = sentence_info.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 
#                                  'SERVING_MIN_OR_MAX_TERM_CODE']).size().unstack(fill_value=0)

# for col in min_max.columns:
#     min_max[col] = min_max[col]/num_distinct_rows
    
# min_max.shape

(722512, 3)

In [116]:
# sentence_info['MAXIMUM_SENTENCE_LENGTH'] = pd.to_numeric(sentence_info['MAXIMUM_SENTENCE_LENGTH'], errors='coerce')
# sentence_info.fillna({'MAXIMUM_SENTENCE_LENGTH': 0, 'MINIMUM_SENTENCE_LENGTH': 0, 
#                       'LENGTH_OF_SUPERVISION':0}, inplace=True)

# others = sentence_info.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX'])\
#         ['MINIMUM_SENTENCE_LENGTH', 'MAXIMUM_SENTENCE_LENGTH', 
#          'LENGTH_OF_SUPERVISION', 'PRIOR_RCD._POINTS/CONVICTIONS'].mean()
    
# others.shape

In [246]:
# all_sentence_vars = pd.concat([counts, counties, offenses, misd_felon, min_max, others], axis=1).reset_index()

Now we want to rejoin these columns onto the admissions data.

In [247]:
# sentence_and_admissions = admissions.merge(all_sentence_vars, on=["INMATE_DOC_NUMBER", "INMATE_COMMITMENT_PREFIX"])


### Triage Version: Apply "Other" To Counties, Offenses Representing < 0.1%, < 0.5% Values

In [148]:
sentence_info.head()

,SENTENCE_COMPONENT_NUMBER,NUMBER_OF_COUNTS,PRIMARY_OFFENSE_CODE,PRIMARY_FELONY/MISDEMEANOR_CD.,COUNTY_OF_CONVICTION_CODE,PRIOR_RCD._POINTS/CONVICTIONS,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SERVING_MIN_OR_MAX_TERM_CODE,SENTENCE_EFFECTIVE(BEGIN)_DATE,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE
0,1,1,SELL SCHEDULE II,FELON,PERQUIMANS,0,0,30000,0.0,MAX.TERM:,1983-07-12 00:00:00,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00,0.0,1983-07-13,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22
1,2,1,SELL SCHEDULE II,FELON,PERQUIMANS,0,0,30000,0.0,MAX.TERM:,1983-07-12 00:00:00,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00,0.0,1983-07-13,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22
2,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,60,90,0.0,MAX.TERM:,1973-01-30 00:00:00,6,AA,1973-01-30 00:00:00,1973-03-28 00:00:00,0.0,1973-01-30,Y,NaN,NaN,MALE,WHITE,1951-07-17 00:00:00,21
3,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24
4,2,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24


In [157]:
def other_under_threshold(df, cols_list, pct_thresholds):
    for idx, col in enumerate(cols_list):
        col_vals = df[col].value_counts()
        col_vals_total = col_vals.sum()
        
        top_vals = col_vals[col_vals/col_vals_total >= pct_thresholds[idx]].index.tolist()
        df.loc[~df[col].isin(top_vals), col] = 'OTHER'


In [158]:
other_under_threshold(sentence_info, cols_list=['PRIMARY_OFFENSE_CODE', 'COUNTY_OF_CONVICTION_CODE'], 
                      pct_thresholds=[0.001, 0.005])

In [159]:
sentence_info.head(5)

,SENTENCE_COMPONENT_NUMBER,NUMBER_OF_COUNTS,PRIMARY_OFFENSE_CODE,PRIMARY_FELONY/MISDEMEANOR_CD.,COUNTY_OF_CONVICTION_CODE,PRIOR_RCD._POINTS/CONVICTIONS,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SERVING_MIN_OR_MAX_TERM_CODE,SENTENCE_EFFECTIVE(BEGIN)_DATE,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE
0,1,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000,0.0,MAX.TERM:,1983-07-12 00:00:00,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00,0.0,1983-07-13,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22
1,2,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000,0.0,MAX.TERM:,1983-07-12 00:00:00,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00,0.0,1983-07-13,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22
2,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,60,90,0.0,MAX.TERM:,1973-01-30 00:00:00,6,AA,1973-01-30 00:00:00,1973-03-28 00:00:00,0.0,1973-01-30,Y,NaN,NaN,MALE,WHITE,1951-07-17 00:00:00,21
3,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24
4,2,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24


In [160]:
len(sentence_info.COUNTY_OF_CONVICTION_CODE.unique())

58

In [143]:
len(sentence_info.PRIMARY_OFFENSE_CODE.unique())

124

In [161]:
sentence_info['MAXIMUM_SENTENCE_LENGTH'] = pd.to_numeric(sentence_info['MAXIMUM_SENTENCE_LENGTH'], errors='coerce')


In [162]:
sentence_info['MINIMUM_SENTENCE_LENGTH'] = pd.to_numeric(sentence_info['MINIMUM_SENTENCE_LENGTH'], errors='coerce')

In [164]:
sentence_info['LENGTH_OF_SUPERVISION'] = pd.to_numeric(sentence_info['LENGTH_OF_SUPERVISION'], errors='coerce')

In [166]:
sentence_info.fillna({'MAXIMUM_SENTENCE_LENGTH': 0, 'MINIMUM_SENTENCE_LENGTH': 0, 
                      'LENGTH_OF_SUPERVISION':0}, inplace=True)

In [167]:
sentence_info.head(10)

,SENTENCE_COMPONENT_NUMBER,NUMBER_OF_COUNTS,PRIMARY_OFFENSE_CODE,PRIMARY_FELONY/MISDEMEANOR_CD.,COUNTY_OF_CONVICTION_CODE,PRIOR_RCD._POINTS/CONVICTIONS,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SERVING_MIN_OR_MAX_TERM_CODE,SENTENCE_EFFECTIVE(BEGIN)_DATE,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE
0,1,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,1983-07-12 00:00:00,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00,0.0,1983-07-13,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22
1,2,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,1983-07-12 00:00:00,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00,0.0,1983-07-13,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22
2,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,60,90.0,0.0,MAX.TERM:,1973-01-30 00:00:00,6,AA,1973-01-30 00:00:00,1973-03-28 00:00:00,0.0,1973-01-30,Y,NaN,NaN,MALE,WHITE,1951-07-17 00:00:00,21
3,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24
4,2,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24
5,3,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24
6,4,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24
7,5,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24
8,6,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24
9,7,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24


In [ ]:
# selected_features = [
#                      # mean
#                      'MINIMUM_SENTENCE_LENGTH', 'MAXIMUM_SENTENCE_LENGTH', 
#                      'LENGTH_OF_SUPERVISION', 'PRIOR_RCD._POINTS/CONVICTIONS',
#                     # sum (aggr)
#                     'DISCI._SEGREGATION_TIME_(DAYS)', 
#                     # sum (categorical)
#                     'DISCIPLINARY_INFRACTION_CODE':'count'
#                     # flag
#                     'SERVING_MIN_OR_MAX_TERM_CODE', 'PRIMARY_FELONY/MISDEMEANOR_CD.', 
#                     'PRIMARY_OFFENSE_CODE', 'COUNTY_OF_CONVICTION_CODE']

In [168]:
# sentence_and_admissions.to_pickle('sentence_and_admissions_triage.pkl')
sentence_info.to_pickle('cleaned_sentence_info.pkl')

In [169]:
# sentence_and_admissions = pd.read_pickle('sentence_and_admissions.pkl')
cleaned_sentence_info = pd.read_pickle('cleaned_sentence_info.pkl')

#### Finalizing Start Dates
If `SENTENCE_BEGIN_DATE_(FOR_MAX)` from the sentence computation table is missing, replace it with `SENTENCE_EFFECTIVE(BEGIN)_DATE` from the sentences table and then `OFFENDER_ADMISSION/INTAKE_DATE` from the court commitment table. If all are null, drop that example.

In [170]:
sentences_table_dates = sentences.groupby(['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX'])\
                          ['SENTENCE_EFFECTIVE(BEGIN)_DATE'].min().to_frame().reset_index()

In [171]:
sentences_table_dates.head()

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,SENTENCE_EFFECTIVE(BEGIN)_DATE
0,1,01,1992-12-14 00:00:00
1,3,01,1988-10-21 00:00:00
2,3,02,2015-02-06 00:00:00
3,4,AA,1983-07-12 00:00:00
4,5,01,1989-08-01 00:00:00


#### LCH to talk to Pedro re: filling dates this way--seems to be lumping multiple sentences

In [178]:
# sentence_and_admissions = sentence_and_admissions.merge(sentences_table_dates, left_on=['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX'],
#                                                  right_on=['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX'])

# sentence_and_admissions.drop(['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX'], axis=1, inplace=True)

sentence_and_admissions = cleaned_sentence_info.merge(sentences_table_dates, 
                                                      left_on=['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 'SENTENCE_EFFECTIVE(BEGIN)_DATE'],
                                                 right_on=['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX', 'SENTENCE_EFFECTIVE(BEGIN)_DATE'])

sentence_and_admissions.drop(['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX'], axis=1, inplace=True)

In [214]:
sentence_and_admissions.head(10)

,SENTENCE_COMPONENT_NUMBER,NUMBER_OF_COUNTS,PRIMARY_OFFENSE_CODE,PRIMARY_FELONY/MISDEMEANOR_CD.,COUNTY_OF_CONVICTION_CODE,PRIOR_RCD._POINTS/CONVICTIONS,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SERVING_MIN_OR_MAX_TERM_CODE,SENTENCE_EFFECTIVE(BEGIN)_DATE,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_BEGIN_DATE_(FOR_MAX),SENTENCE_END,PAROLE_DAYS,OFFENDER_ADMISSION/INTAKE_DATE,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE,SENTENCE_START
0,1,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,1983-07-12 00:00:00,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00,0.0,1983-07-13,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22,0001-01-01 00:00:00
1,2,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,1983-07-12 00:00:00,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00,0.0,1983-07-13,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22,0001-01-01 00:00:00
2,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,60,90.0,0.0,MAX.TERM:,1973-01-30 00:00:00,6,AA,1973-01-30 00:00:00,1973-03-28 00:00:00,0.0,1973-01-30,Y,NaN,NaN,MALE,WHITE,1951-07-17 00:00:00,21,1973-01-30 00:00:00
3,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24,1973-04-11 00:00:00
4,2,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24,1973-04-11 00:00:00
5,3,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24,1973-04-11 00:00:00
6,4,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24,1973-04-11 00:00:00
7,5,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24,1973-04-11 00:00:00
8,6,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24,1973-04-11 00:00:00
9,7,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,1973-04-11 00:00:00,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00,0.0,1973-04-15,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24,1973-04-11 00:00:00


In [180]:
sentence_and_admissions['SENTENCE_START'] = sentence_and_admissions['SENTENCE_BEGIN_DATE_(FOR_MAX)']\
                                .fillna(sentence_and_admissions['SENTENCE_EFFECTIVE(BEGIN)_DATE'])\
                                .fillna(sentence_and_admissions['OFFENDER_ADMISSION/INTAKE_DATE'])                            

In [181]:
clean_dates = sentence_and_admissions[sentence_and_admissions['SENTENCE_START'].notnull()]

In [182]:
clean_dates.drop(['SENTENCE_BEGIN_DATE_(FOR_MAX)', 'SENTENCE_EFFECTIVE(BEGIN)_DATE', 'OFFENDER_ADMISSION/INTAKE_DATE'], 
                axis=1, inplace=True)

In [183]:
clean_dates.head(3)

,SENTENCE_COMPONENT_NUMBER,NUMBER_OF_COUNTS,PRIMARY_OFFENSE_CODE,PRIMARY_FELONY/MISDEMEANOR_CD.,COUNTY_OF_CONVICTION_CODE,PRIOR_RCD._POINTS/CONVICTIONS,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SERVING_MIN_OR_MAX_TERM_CODE,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_END,PAROLE_DAYS,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE,SENTENCE_START
0,1,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,4,AA,1984-07-11 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22,0001-01-01 00:00:00
1,2,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,4,AA,1984-07-11 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22,0001-01-01 00:00:00
2,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,60,90.0,0.0,MAX.TERM:,6,AA,1973-03-28 00:00:00,0.0,Y,NaN,NaN,MALE,WHITE,1951-07-17 00:00:00,21,1973-01-30 00:00:00


In [185]:
# clean_dates.to_pickle('clean_dates.pkl')
clean_dates.to_pickle('clean_dates_triage.pkl')

In [2]:
# clean_dates = pd.read_pickle('clean_dates.pkl')
clean_dates = pd.read_pickle('clean_dates_triage.pkl')

### Bringing In Disciplinary Data

Now I will bring in information about disciplinary actions. I will once again aggregate all the infractions that occur less than 0.1% of the time into the category "OTHER", giving us 54 infraction categories overall. 

In [186]:
just_dates = clean_dates[['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 'SENTENCE_START', 'SENTENCE_END']]

In [189]:
just_dates.head()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_START,SENTENCE_END
0,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00
1,4,AA,0001-01-01 00:00:00,1984-07-11 00:00:00
2,6,AA,1973-01-30 00:00:00,1973-03-28 00:00:00
3,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00
4,6,AB,1973-04-11 00:00:00,1975-08-18 00:00:00


In [258]:
# Infractions grouping
# infraction_percentages = discipline["DISCIPLINARY_INFRACTION_CODE"].value_counts(normalize=True, dropna=False)
# infraction_groupings = {i:i for i in discipline["DISCIPLINARY_INFRACTION_CODE"].unique()}

# for i, j in infraction_percentages[infraction_percentages < 0.001].iteritems():
#     infraction_groupings[i] = "OTHER"

In [259]:
# discipline["DISCIPLINARY_INFRACTION"] = [infraction_groupings[j] for i,j in discipline["DISCIPLINARY_INFRACTION_CODE"].iteritems()]

In [187]:
other_under_threshold(discipline, cols_list=['DISCIPLINARY_INFRACTION_CODE'], 
                      pct_thresholds=[0.001])

In [188]:
len(discipline.DISCIPLINARY_INFRACTION_CODE.unique())

56

In [190]:
dates_and_infractions = just_dates.merge(discipline, on='INMATE_DOC_NUMBER')

In [191]:
dates_and_infractions.shape

(15860592, 7)

In [192]:
dates_and_infractions = dates_and_infractions[
    (dates_and_infractions['DISCIPLINARY_INFRACTION_DATE'] >= dates_and_infractions['SENTENCE_START']) &
    (dates_and_infractions['DISCIPLINARY_INFRACTION_DATE'] <= dates_and_infractions['SENTENCE_END'])
]

In [193]:
dates_and_infractions.shape

(5592881, 7)

In [194]:
dates_and_infractions.head()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_START,SENTENCE_END,DISCIPLINARY_INFRACTION_DATE,DISCIPLINARY_INFRACTION_CODE,DISCI._SEGREGATION_TIME_(DAYS)
0,10,AA,1975-06-11 00:00:00,1977-03-17 00:00:00,1975-07-26,MISUSE SUPPLIES,0.0
1,10,AA,1975-06-11 00:00:00,1977-03-17 00:00:00,1975-07-28,DISOBEY ORDER,0.0
2,10,AA,1975-06-11 00:00:00,1977-03-17 00:00:00,1975-11-23,DISOBEY ORDER,0.0
3,10,AA,1975-06-11 00:00:00,1977-03-17 00:00:00,1976-01-03,DISOBEY ORDER,0.0
4,10,AA,1975-06-11 00:00:00,1977-03-17 00:00:00,1976-01-22,DISOBEY ORDER,0.0


In [195]:
# basic_vars = dates_and_infractions.groupby(
#         ['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX'])\
#         .agg({'DISCI._SEGREGATION_TIME_(DAYS)':'sum', 'DISCIPLINARY_INFRACTION_CODE':'count'})
    
# basic_vars.rename({'DISCIPLINARY_INFRACTION_CODE': 'INFRAC_COUNT'}, axis=1, inplace=True)
# basic_vars

DISCI._SEGREGATION_TIME_(DAYS)  \
INMATE_DOC_NUMBER INMATE_COMMITMENT_PREFIX                                   
10                AA                                                   0.0   
                  AB                                                   0.0   
26                AA                                                   0.0   
31                AA                                                   0.0   
33                AD                                                   0.0   
                  BA                                                  60.0   
35                BA                                                  20.0   
                  BB                                                 390.0   
                  BC                                                1935.0   
37                BA                                                  90.0   
39                AA                                                   0.0   
50                AA                                                   0.0   
54                AA                                                   0.0   
73                AA                                                   0.0   
74                AB                                                   0.0   
75                AA                                                   0.0   
78                AB                                                   0.0   
80                BA                                                   0.0   
81                AA                                                   0.0   
87                AA                                                   0.0   
                  AB                                                   0.0   
93                AA                                                   0.0   
104               AA                                                   0.0   
114               BA                                                 130.0   
                  BB                                                  30.0   
115               AA                                                   0.0   
118               AB                                                   0.0   
121               AA                                                   0.0   
130               BA                                                   0.0   
133               BA                                                1020.0   
142               AA                                                   0.0   
                  AB                                                   0.0   
147               AA                                                   0.0   
                  AB                                                   0.0   
                  AD                                                   0.0   
149               AA                                                   0.0   
156               BA                                                   0.0   
164               AA                                                   0.0   
167               AA                                                   0.0   
                  BA                                                 345.0   
170               AA                                                   0.0   
171               AA                                                   0.0   
181               BA                                                   0.0   
190               AA                                                   0.0   
192               AA                                                   0.0   
                  AB                                                   0.0   
                  AC                                                   0.0   
                  BA                                                  30.0   
203               BA                                                 120.0   
204               AA                                              

In [265]:
# infractions = dates_and_infractions.groupby(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 
#                                   'DISCIPLINARY_INFRACTION']).size().unstack(fill_value=0)

# infractions.columns = ["INFRAC_" + str(i) for i in infractions.columns]
# for col in infractions.columns:
#     infractions[col] = infractions[col]/basic_vars['INFRAC_COUNT']
    
# infractions.shape

(363598, 56)

In [266]:
# all_infractions_data = pd.concat([basic_vars, infractions], axis=1)
# all_infractions_data = all_infractions_data.reset_index()

In [267]:
# all_infractions_data.head()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,DISCI._SEGREGATION_TIME_(DAYS),INFRAC_COUNT,INFRAC_ACTIVE RIOTER,INFRAC_ASSAULT PERSON W/WEAPON,INFRAC_ASSAULT STAFF W/WEAPON,INFRAC_ASSAULT STAFF/THROWING LIQUIDS,INFRAC_ASSLT OTHER W/UNLIKELY INJ,INFRAC_ASSLT STAFF W/UNLIKELY INJ,INFRAC_ATTEMPT CLASS A OFFENSE,INFRAC_ATTEMPT CLASS B OFFENSE,INFRAC_ATTEMPT CLASS C OFFENSE,INFRAC_ATTEMPT CLASS D OFFENSE,INFRAC_BARTER/TRADE/LOAN MONEY,INFRAC_CREATE OFFENSIVE CONDITION,INFRAC_DAMAGE STATE/ANOTHERS PROPERTY,INFRAC_DISOBEY ORDER,INFRAC_ESCAPE,INFRAC_FAKE ILLNESS,INFRAC_FALSE ALLEGATIONS ON STAFF,INFRAC_FIGHT W/WEAPON OR REQ.OUT.MED,INFRAC_FIGHTING,INFRAC_FLOOD CELL,INFRAC_GAMBLING,INFRAC_HIGH RISK ACT,INFRAC_ILLEGAL CLOTH/LINEN/SHEETS,INFRAC_INTERFERE W/STAFF,INFRAC_INVOLVEMENT W/GANG OR SRG,INFRAC_LEAVE\QUIT COMM BASED PROGRAM,INFRAC_LOCK TAMPERING,INFRAC_MISUSE MEDICINE,INFRAC_MISUSE SUPPLIES,INFRAC_MISUSE/UNAUTH-USE PHONE/MAIL,INFRAC_NEGLIGENTLY PERFORM DUTIES,INFRAC_NO THREAT CONTRABAND,INFRAC_OFFER/ACCEPT BRIBE STAFF,INFRAC_OTHER,INFRAC_POSS AUDIO/VIDEO/IMAGE DEVICE,INFRAC_POSS MONEY/UNAUTHORIZED FUNDS,INFRAC_POSSESS EXCESS STAMPS,INFRAC_PROFANE LANGUAGE,INFRAC_PROPERTY TAMPERING,INFRAC_PROVOKE ASSAULT,INFRAC_REFUSE SUBMIT/DRUG/BREATH TEST,INFRAC_SELF INJURY,INFRAC_SELL/MISUSE MEDICATION,INFRAC_SET A FIRE,INFRAC_SEXUAL ACT,INFRAC_SUBSTANCE POSSESSION,INFRAC_THEFT OF PROPERTY,INFRAC_THREATEN TO HARM/INJURE STAFF,INFRAC_UNAUTH TOBACCO NON-PERSNL USE,INFRAC_UNAUTHORIZED FUNDS,INFRAC_UNAUTHORIZED LEAVE,INFRAC_UNAUTHORIZED LOCATION,INFRAC_UNAUTHORIZED TOBACCO USE,INFRAC_UNKEMPT ROOM,INFRAC_VERBAL THREAT,INFRAC_WEAPON POSSESSION
0,10,AA,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.545455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.090909,0.090909,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909
1,10,AB,0.0,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.466667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.066667,0.133333,0.0,0.0,0.0,0.0,0.0,0.000000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667
2,26,AA,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,31,AA,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,33,AD,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [196]:
# Left join disciplinary data onto all other data to include those without any disciplinary actions
# all_tables_in = clean_dates.merge(all_infractions_data, on=['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX'],
#                                  how='left')
all_tables_in_triage = clean_dates.merge(dates_and_infractions, on=['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 'SENTENCE_START', 'SENTENCE_END'],
                                 how='left')


In [197]:
all_tables_in_triage.columns

Index(['SENTENCE_COMPONENT_NUMBER', 'NUMBER_OF_COUNTS', 'PRIMARY_OFFENSE_CODE',
       'PRIMARY_FELONY/MISDEMEANOR_CD.', 'COUNTY_OF_CONVICTION_CODE',
       'PRIOR_RCD._POINTS/CONVICTIONS', 'MINIMUM_SENTENCE_LENGTH',
       'MAXIMUM_SENTENCE_LENGTH', 'LENGTH_OF_SUPERVISION',
       'SERVING_MIN_OR_MAX_TERM_CODE', 'INMATE_DOC_NUMBER',
       'INMATE_COMMITMENT_PREFIX', 'SENTENCE_END', 'PAROLE_DAYS',
       'NEW_PERIOD_OF_INCARCERATION_FL', 'P&P_COMMITMENT_STATUS_FLAG',
       'NEW_PERIOD_OF_SUPERVISION_FLAG', 'INMATE_GENDER_CODE',
       'INMATE_RACE_CODE', 'INMATE_BIRTH_DATE', 'AGE_AT_RELEASE',
       'SENTENCE_START', 'DISCIPLINARY_INFRACTION_DATE',
       'DISCIPLINARY_INFRACTION_CODE', 'DISCI._SEGREGATION_TIME_(DAYS)'],
      dtype='object')

In [269]:
# all_tables_in[all_infractions_data.columns] = all_tables_in[all_infractions_data.columns].fillna(value=0)

In [198]:
# all_tables_in.to_pickle('all_tables_in.pkl')
all_tables_in_triage.to_pickle('data/preprocessed/all_tables_in_triage.pkl')

In [9]:
# all_tables_in = pd.read_pickle('data/preprocessed/all_tables_in.pkl')
all_tables_in_triage = pd.read_pickle('data/preprocessed/all_tables_in_triage.pkl')

### Getting Number of Previous Commitments

Window size should be larger than the max number of commitments for any individual.

In [202]:
# max_commitments = max(with_indicators.groupby('INMATE_DOC_NUMBER')['INMATE_COMMITMENT_PREFIX'].count())
max_commitments = max(all_tables_in_triage.groupby('INMATE_DOC_NUMBER')['INMATE_COMMITMENT_PREFIX'].count())

In [14]:
# previous_commit = with_indicators.groupby('INMATE_DOC_NUMBER')['INMATE_COMMITMENT_PREFIX']\
#                                  .rolling(window = max_commitments + 1).count()

In [204]:
previous_commit = all_tables_in_triage.groupby('INMATE_DOC_NUMBER')['INMATE_COMMITMENT_PREFIX']\
                                 .rolling(window = max_commitments + 1).count()

In [205]:
previous_commit_df = previous_commit.to_frame().reset_index().drop('level_1', axis=1)\
                       .rename({'INMATE_COMMITMENT_PREFIX': 'NUM_PRIOR_COMMITMENTS'}, axis=1)

In [206]:
# with_indicators['PREVIOUS_COMMITMENTS'] = previous_commit_df['NUM_PRIOR_COMMITMENTS']
# with_indicators.head()
all_tables_in_triage['PREVIOUS_COMMITMENTS'] = previous_commit_df['NUM_PRIOR_COMMITMENTS']
all_tables_in_triage.head()

,SENTENCE_COMPONENT_NUMBER,NUMBER_OF_COUNTS,PRIMARY_OFFENSE_CODE,PRIMARY_FELONY/MISDEMEANOR_CD.,COUNTY_OF_CONVICTION_CODE,PRIOR_RCD._POINTS/CONVICTIONS,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SERVING_MIN_OR_MAX_TERM_CODE,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_END,PAROLE_DAYS,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE,SENTENCE_START,DISCIPLINARY_INFRACTION_DATE,DISCIPLINARY_INFRACTION_CODE,DISCI._SEGREGATION_TIME_(DAYS),PREVIOUS_COMMITMENTS
0,1,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,4,AA,1984-07-11 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22,0001-01-01 00:00:00,NaT,NaN,NaN,1.0
1,2,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,4,AA,1984-07-11 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22,0001-01-01 00:00:00,NaT,NaN,NaN,2.0
2,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,60,90.0,0.0,MAX.TERM:,6,AA,1973-03-28 00:00:00,0.0,Y,NaN,NaN,MALE,WHITE,1951-07-17 00:00:00,21,1973-01-30 00:00:00,NaT,NaN,NaN,1.0
3,1,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,6,AB,1975-08-18 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24,1973-04-11 00:00:00,NaT,NaN,NaN,2.0
4,2,1,WORTHLESS CHECK,MISD.,CUMBERLAND,0,30,0.0,0.0,MIN.TERM:,6,AB,1975-08-18 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,WHITE,1951-07-17 00:00:00,24,1973-04-11 00:00:00,NaT,NaN,NaN,3.0


In [208]:
all_tables_in_triage.shape

(16336288, 26)

In [207]:
# with_indicators.to_pickle('with_indicators.pkl')
all_tables_in_triage.to_pickle('data/preprocessed/with_commitments_triage.pkl')

In [ ]:
# all_tables_in_triage = pd.read_pickle('data/preprocessed/with_commitments_triage.pkl')

## Set Recidivated Flag

Finally, I will make the target variable, recidivated. Here I will define recidivism as another commitment beginning between 3 months and 3 years after the end of the previous. This 3 month gap is to attempt to filter out some commitments which appear to be served consecutively with small recorded gaps in between.

The commitments are filtered by ID and then by commitment, so we can just loop over the rows and compare with 1 row back.

In [209]:
# just_dates = with_indicators[['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 'SENTENCE_START', 'SENTENCE_END']]
just_dates = all_tables_in_triage[['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 'SENTENCE_START', 'SENTENCE_END']]

In [213]:
all_tables_in_triage.loc[all_tables_in_triage['SENTENCE_START'] < datetime.strptime('1800-01-01', '%Y-%m-%d')]

,SENTENCE_COMPONENT_NUMBER,NUMBER_OF_COUNTS,PRIMARY_OFFENSE_CODE,PRIMARY_FELONY/MISDEMEANOR_CD.,COUNTY_OF_CONVICTION_CODE,PRIOR_RCD._POINTS/CONVICTIONS,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SERVING_MIN_OR_MAX_TERM_CODE,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_END,PAROLE_DAYS,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE,SENTENCE_START,DISCIPLINARY_INFRACTION_DATE,DISCIPLINARY_INFRACTION_CODE,DISCI._SEGREGATION_TIME_(DAYS),PREVIOUS_COMMITMENTS
0,1,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,4,AA,1984-07-11 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22,0001-01-01 00:00:00,NaT,NaN,NaN,1.0
1,2,1,SELL SCHEDULE II,FELON,OTHER,0,0,30000.0,0.0,MAX.TERM:,4,AA,1984-07-11 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,WHITE,1961-10-15 00:00:00,22,0001-01-01 00:00:00,NaT,NaN,NaN,2.0
43,2,1,B & E & L,FELON,NEW HANOVER,0,100000,0.0,0.0,MIN.TERM:,10,AB,1983-06-27 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,BLACK,1953-05-18 00:00:00,30,0001-01-01 00:00:00,1975-07-26,MISUSE SUPPLIES,0.0,12.0
44,2,1,B & E & L,FELON,NEW HANOVER,0,100000,0.0,0.0,MIN.TERM:,10,AB,1983-06-27 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,BLACK,1953-05-18 00:00:00,30,0001-01-01 00:00:00,1975-07-28,DISOBEY ORDER,0.0,13.0
45,2,1,B & E & L,FELON,NEW HANOVER,0,100000,0.0,0.0,MIN.TERM:,10,AB,1983-06-27 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,BLACK,1953-05-18 00:00:00,30,0001-01-01 00:00:00,1975-11-23,DISOBEY ORDER,0.0,14.0
46,2,1,B & E & L,FELON,NEW HANOVER,0,100000,0.0,0.0,MIN.TERM:,10,AB,1983-06-27 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,BLACK,1953-05-18 00:00:00,30,0001-01-01 00:00:00,1976-01-03,DISOBEY ORDER,0.0,15.0
47,2,1,B & E & L,FELON,NEW HANOVER,0,100000,0.0,0.0,MIN.TERM:,10,AB,1983-06-27 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,BLACK,1953-05-18 00:00:00,30,0001-01-01 00:00:00,1976-01-22,DISOBEY ORDER,0.0,16.0
48,2,1,B & E & L,FELON,NEW HANOVER,0,100000,0.0,0.0,MIN.TERM:,10,AB,1983-06-27 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,BLACK,1953-05-18 00:00:00,30,0001-01-01 00:00:00,1976-01-22,PROPERTY TAMPERING,0.0,17.0
49,2,1,B & E & L,FELON,NEW HANOVER,0,100000,0.0,0.0,MIN.TERM:,10,AB,1983-06-27 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,BLACK,1953-05-18 00:00:00,30,0001-01-01 00:00:00,1976-03-07,DISOBEY ORDER,0.0,18.0
50,2,1,B & E & L,FELON,NEW HANOVER,0,100000,0.0,0.0,MIN.TERM:,10,AB,1983-06-27 00:00:00,0.0,Y,NORMAL NORM,NaN,MALE,BLACK,1953-05-18 00:00:00,30,0001-01-01 00:00:00,1976-04-08,DISOBEY ORDER,0.0,19.0


In [21]:
just_dates.tail()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_START,SENTENCE_END
854061,1566187,BA,2018-01-10 00:00:00,2018-02-22 00:00:00
854062,1566198,BA,2018-12-17 00:00:00,2019-10-06 00:00:00
854063,1566234,BA,2019-01-14 00:00:00,2019-06-26 00:00:00
854064,1566265,BA,2018-07-23 00:00:00,2018-10-01 00:00:00
854065,1566292,BA,2018-01-02 00:00:00,2018-03-28 00:00:00


In [22]:
just_dates.tail()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_START,SENTENCE_END
854061,1566187,BA,2018-01-10 00:00:00,2018-02-22 00:00:00
854062,1566198,BA,2018-12-17 00:00:00,2019-10-06 00:00:00
854063,1566234,BA,2019-01-14 00:00:00,2019-06-26 00:00:00
854064,1566265,BA,2018-07-23 00:00:00,2018-10-01 00:00:00
854065,1566292,BA,2018-01-02 00:00:00,2018-03-28 00:00:00


In [23]:
# with_indicators.head(3)
all_tables_in_triage.head(3)

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_END,PAROLE_DAYS,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE,NUMBER_OF_COUNTS,COUNTY_ALAMANCE,COUNTY_BEAUFORT,COUNTY_BRUNSWICK,COUNTY_BUNCOMBE,COUNTY_BURKE,COUNTY_CABARRUS,COUNTY_CALDWELL,COUNTY_CARTERET,COUNTY_CATAWBA,COUNTY_CLEVELAND,COUNTY_COLUMBUS,COUNTY_CRAVEN,COUNTY_CUMBERLAND,COUNTY_DAVIDSON,COUNTY_DUPLIN,COUNTY_DURHAM,COUNTY_EDGECOMBE,COUNTY_FORSYTH,COUNTY_FRANKLIN,COUNTY_GASTON,COUNTY_GRANVILLE,COUNTY_GUILFORD,COUNTY_HALIFAX,COUNTY_HARNETT,COUNTY_HAYWOOD,COUNTY_HENDERSON,COUNTY_HOKE,COUNTY_IREDELL,COUNTY_JOHNSTON,COUNTY_LEE,COUNTY_LENOIR,COUNTY_LINCOLN,COUNTY_MCDOWELL,COUNTY_MECKLENBURG,COUNTY_MOORE,COUNTY_NASH,COUNTY_NEW HANOVER,COUNTY_ONSLOW,COUNTY_ORANGE,COUNTY_OTHER,COUNTY_PERSON,COUNTY_PITT,COUNTY_RANDOLPH,COUNTY_RICHMOND,COUNTY_ROBESON,COUNTY_ROCKINGHAM,COUNTY_ROWAN,COUNTY_RUTHERFORD,COUNTY_SAMPSON,COUNTY_SCOTLAND,COUNTY_STANLY,COUNTY_SURRY,COUNTY_UNION,COUNTY_VANCE,COUNTY_WAKE,COUNTY_WAYNE,COUNTY_WILKES,COUNTY_WILSON,OFFENSE_3M REIMPRISON VIOL,OFFENSE_ABANDONMENT,OFFENSE_ARMED ROBBERY,OFFENSE_ASSAULT,OFFENSE_ASSAULT BY STRANGULATION,OFFENSE_ASSAULT INFLICT SERI BODY INJ,OFFENSE_ASSAULT ISI,OFFENSE_ASSAULT ON FEMALE,OFFENSE_ASSAULT ON OFFICER/ST EMPLOYEE,OFFENSE_ASSAULT ON POLICEMAN,OFFENSE_ASSAULT ON PUBLIC OFFICIAL,OFFENSE_AWDW,OFFENSE_AWDW GOV OFFICERS/EMPLOYEES,OFFENSE_AWDWISI,OFFENSE_AWDWWITK,OFFENSE_AWDWWITKISI,OFFENSE_B & E & L,OFFENSE_B & E VEHICLES,OFFENSE_BURGLARY 1ST DEGREE,OFFENSE_BURGLARY 2ND DEGREE,OFFENSE_CARRY CONCEALED WEAPON,OFFENSE_CHEAT - PROPERTY/SERVICES,OFFENSE_COMMON LAW FORGERY,OFFENSE_COMMON LAW ROBBERY,OFFENSE_COMMUNICATING THREATS,OFFENSE_CREDIT CARD THEFT,OFFENSE_CRIME AGAINST NATURE,OFFENSE_DAMAGE TO PROPERTY,OFFENSE_DEL/SELL SCHEDULE II,OFFENSE_DISCHG FIREARM-OCC PROPERTY,OFFENSE_DISORDERLY CONDUCT,OFFENSE_DRIV LICENSE PERM RVK,OFFENSE_DRIV LICENSE REVOKED,OFFENSE_DRIVING UNDER INFLUENCE (DUI),OFFENSE_DRUG PARA - USE/POSSESS,OFFENSE_DRUNK & DISORDERLY,OFFENSE_DWI DRIVING WHILE IMPAIRED,OFFENSE_DWI LEVEL 1,OFFENSE_DWI LEVEL 1 AGGRAVATED,OFFENSE_DWI LEVEL 2,OFFENSE_DWI LEVEL 3,OFFENSE_DWI LEVEL 4,OFFENSE_DWI LEVEL 5,OFFENSE_EMBEZZLEMENT,OFFENSE_EMPLOY SEC VIOLATION,OFFENSE_ESCAPE PRISON,OFFENSE_FAIL TO REGISTER (SEX OFFENDER,OFFENSE_FAILURE TO STOP FOR ACCIDENT,OFFENSE_FELONY B&E,OFFENSE_FINANCIAL CARD FRAUD/MISD,OFFENSE_FORGERY,OFFENSE_FORGERY AND UTTERING,OFFENSE_HABITUAL FELON,OFFENSE_HABITUAL IMPAIRED DRIVING,OFFENSE_HABITUAL MISDEMEANOR ASSAULT,OFFENSE_HIT AND RUN,OFFENSE_IDENTITY FRAUD/THEFT,OFFENSE_INDECENT LIBERTY W/CHILD,OFFENSE_INVOLUNTARY MANSLAUGHTER,OFFENSE_KIDNAPPING 1ST DEGREE,OFFENSE_KIDNAPPING 2ND DEGREE,OFFENSE_LARCENY,OFFENSE_LARCENY (OVER $200),OFFENSE_LARCENY AFTER B & E,OFFENSE_LARCENY AND RECEIVING,OFFENSE_LARCENY BY SERVANT/EMPLOYEE,OFFENSE_LARCENY FROM PERSON,OFFENSE_LARCENY OF FIREARMS,OFFENSE_LARCENY OF MOTOR VEHICLE,OFFENSE_LARCENY OVER $1000,OFFENSE_MAINT ANY PLACE CONTR SUB,OFFENSE_MANSLAUGHTER,OFFENSE_MISD B&E,OFFENSE_MURDER FIRST DEGREE,OFFENSE_MURDER SECOND DEGREE,OFFENSE_NON-SUPPORT,OFFENSE_OBSTRUCTING JUSTICE,OFFENSE_OBT PROP BY FALSE PR/CHTS/SER,OFFENSE_OBTAIN CONTR SUBST BY FRAUD,OFFENSE_OPERATE VEHICLE W/O LICENSE,OFFENSE_OTHER,OFFENSE_OTHER MISDEMEANANT,OFFENSE_POSSESS SCHEDULE I,OFFENSE_POSSESS SCHEDULE II,OFFENSE_POSSESS SCHEDULE VI,OFFENSE_POSSESS WIT SELL CONTROL SUBST,OFFENSE_POSSESS WITS SCHEDULE I,OFFENSE_POSSESS WITS SCHEDULE II,OFFENSE_POSSESS WITS SCHEDULE VI,OFFENSE_POSSESSING STOLEN GOODS,OFFENSE_POSSESSION OF FIREARM BY FELON,OFFENSE_POST RELEASE REVOCATION,OFFENSE_RAPE FIRST DEGREE,OFFENSE_RAPE SECOND DEGREE,OFFENSE_RECEIVING STOLEN GOODS,OFFENSE_RECEIVING STOLEN VEHICLE,OFFENSE_RECKLESS DRIVING,OFFENSE_RESISTING OFFICER,OFFENSE_ROBBERY W/DANGEROUS WEAPON,OFFENSE_SECOND DEGREE TRESPASS,OFFENSE_SELL CONTROL SUBSTANCE,OFFENSE_SELL SCHEDULE I,OFFENSE_SELL SCHEDULE II,OFFE

In [128]:
# final_data = with_indicators.loc[:,:]
final_data_no_preprocess = all_tables_in.loc[:,:]

In [129]:
final_data_no_preprocess['RECIDIVATED'] = recidivated

In [130]:
final_data_no_preprocess.head()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_END,PAROLE_DAYS,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE,NUMBER_OF_COUNTS,COUNTY_ALAMANCE,COUNTY_BEAUFORT,COUNTY_BRUNSWICK,COUNTY_BUNCOMBE,COUNTY_BURKE,COUNTY_CABARRUS,COUNTY_CALDWELL,COUNTY_CARTERET,COUNTY_CATAWBA,COUNTY_CLEVELAND,COUNTY_COLUMBUS,COUNTY_CRAVEN,COUNTY_CUMBERLAND,COUNTY_DAVIDSON,COUNTY_DUPLIN,COUNTY_DURHAM,COUNTY_EDGECOMBE,COUNTY_FORSYTH,COUNTY_FRANKLIN,COUNTY_GASTON,COUNTY_GRANVILLE,COUNTY_GUILFORD,COUNTY_HALIFAX,COUNTY_HARNETT,COUNTY_HAYWOOD,COUNTY_HENDERSON,COUNTY_HOKE,COUNTY_IREDELL,COUNTY_JOHNSTON,COUNTY_LEE,COUNTY_LENOIR,COUNTY_LINCOLN,COUNTY_MCDOWELL,COUNTY_MECKLENBURG,COUNTY_MOORE,COUNTY_NASH,COUNTY_NEW HANOVER,COUNTY_ONSLOW,COUNTY_ORANGE,COUNTY_OTHER,COUNTY_PERSON,COUNTY_PITT,COUNTY_RANDOLPH,COUNTY_RICHMOND,COUNTY_ROBESON,COUNTY_ROCKINGHAM,COUNTY_ROWAN,COUNTY_RUTHERFORD,COUNTY_SAMPSON,COUNTY_SCOTLAND,COUNTY_STANLY,COUNTY_SURRY,COUNTY_UNION,COUNTY_VANCE,COUNTY_WAKE,COUNTY_WAYNE,COUNTY_WILKES,COUNTY_WILSON,OFFENSE_3M REIMPRISON VIOL,OFFENSE_ABANDONMENT,OFFENSE_ARMED ROBBERY,OFFENSE_ASSAULT,OFFENSE_ASSAULT BY STRANGULATION,OFFENSE_ASSAULT INFLICT SERI BODY INJ,OFFENSE_ASSAULT ISI,OFFENSE_ASSAULT ON FEMALE,OFFENSE_ASSAULT ON OFFICER/ST EMPLOYEE,OFFENSE_ASSAULT ON POLICEMAN,OFFENSE_ASSAULT ON PUBLIC OFFICIAL,OFFENSE_AWDW,OFFENSE_AWDW GOV OFFICERS/EMPLOYEES,OFFENSE_AWDWISI,OFFENSE_AWDWWITK,OFFENSE_AWDWWITKISI,OFFENSE_B & E & L,OFFENSE_B & E VEHICLES,OFFENSE_BURGLARY 1ST DEGREE,OFFENSE_BURGLARY 2ND DEGREE,OFFENSE_CARRY CONCEALED WEAPON,OFFENSE_CHEAT - PROPERTY/SERVICES,OFFENSE_COMMON LAW FORGERY,OFFENSE_COMMON LAW ROBBERY,OFFENSE_COMMUNICATING THREATS,OFFENSE_CREDIT CARD THEFT,OFFENSE_CRIME AGAINST NATURE,OFFENSE_DAMAGE TO PROPERTY,OFFENSE_DEL/SELL SCHEDULE II,OFFENSE_DISCHG FIREARM-OCC PROPERTY,OFFENSE_DISORDERLY CONDUCT,OFFENSE_DRIV LICENSE PERM RVK,OFFENSE_DRIV LICENSE REVOKED,OFFENSE_DRIVING UNDER INFLUENCE (DUI),OFFENSE_DRUG PARA - USE/POSSESS,OFFENSE_DRUNK & DISORDERLY,OFFENSE_DWI DRIVING WHILE IMPAIRED,OFFENSE_DWI LEVEL 1,OFFENSE_DWI LEVEL 1 AGGRAVATED,OFFENSE_DWI LEVEL 2,OFFENSE_DWI LEVEL 3,OFFENSE_DWI LEVEL 4,OFFENSE_DWI LEVEL 5,OFFENSE_EMBEZZLEMENT,OFFENSE_EMPLOY SEC VIOLATION,OFFENSE_ESCAPE PRISON,OFFENSE_FAIL TO REGISTER (SEX OFFENDER,OFFENSE_FAILURE TO STOP FOR ACCIDENT,OFFENSE_FELONY B&E,OFFENSE_FINANCIAL CARD FRAUD/MISD,OFFENSE_FORGERY,OFFENSE_FORGERY AND UTTERING,OFFENSE_HABITUAL FELON,OFFENSE_HABITUAL IMPAIRED DRIVING,OFFENSE_HABITUAL MISDEMEANOR ASSAULT,OFFENSE_HIT AND RUN,OFFENSE_IDENTITY FRAUD/THEFT,OFFENSE_INDECENT LIBERTY W/CHILD,OFFENSE_INVOLUNTARY MANSLAUGHTER,OFFENSE_KIDNAPPING 1ST DEGREE,OFFENSE_KIDNAPPING 2ND DEGREE,OFFENSE_LARCENY,OFFENSE_LARCENY (OVER $200),OFFENSE_LARCENY AFTER B & E,OFFENSE_LARCENY AND RECEIVING,OFFENSE_LARCENY BY SERVANT/EMPLOYEE,OFFENSE_LARCENY FROM PERSON,OFFENSE_LARCENY OF FIREARMS,OFFENSE_LARCENY OF MOTOR VEHICLE,OFFENSE_LARCENY OVER $1000,OFFENSE_MAINT ANY PLACE CONTR SUB,OFFENSE_MANSLAUGHTER,OFFENSE_MISD B&E,OFFENSE_MURDER FIRST DEGREE,OFFENSE_MURDER SECOND DEGREE,OFFENSE_NON-SUPPORT,OFFENSE_OBSTRUCTING JUSTICE,OFFENSE_OBT PROP BY FALSE PR/CHTS/SER,OFFENSE_OBTAIN CONTR SUBST BY FRAUD,OFFENSE_OPERATE VEHICLE W/O LICENSE,OFFENSE_OTHER,OFFENSE_OTHER MISDEMEANANT,OFFENSE_POSSESS SCHEDULE I,OFFENSE_POSSESS SCHEDULE II,OFFENSE_POSSESS SCHEDULE VI,OFFENSE_POSSESS WIT SELL CONTROL SUBST,OFFENSE_POSSESS WITS SCHEDULE I,OFFENSE_POSSESS WITS SCHEDULE II,OFFENSE_POSSESS WITS SCHEDULE VI,OFFENSE_POSSESSING STOLEN GOODS,OFFENSE_POSSESSION OF FIREARM BY FELON,OFFENSE_POST RELEASE REVOCATION,OFFENSE_RAPE FIRST DEGREE,OFFENSE_RAPE SECOND DEGREE,OFFENSE_RECEIVING STOLEN GOODS,OFFENSE_RECEIVING STOLEN VEHICLE,OFFENSE_RECKLESS DRIVING,OFFENSE_RESISTING OFFICER,OFFENSE_ROBBERY W/DANGEROUS WEAPON,OFFENSE_SECOND DEGREE TRESPASS,OFFENSE_SELL CONTROL SUBSTANCE,OFFENSE_SELL SCHEDULE I,OFFENSE_SELL SCHEDULE II,OFFE

In [147]:
final_data.loc[(final_data['SENTENCE_END'].isna()) | (final_data['INMATE_BIRTH_DATE'].isna()) | (final_data['SENTENCE_START'].isna())]

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_END,PAROLE_DAYS,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE,NUMBER_OF_COUNTS,OFFENSE_3M REIMPRISON VIOL,OFFENSE_ABANDONMENT,OFFENSE_ARMED ROBBERY,OFFENSE_ASSAULT,OFFENSE_ASSAULT BY STRANGULATION,OFFENSE_ASSAULT INFLICT SERI BODY INJ,OFFENSE_ASSAULT ISI,OFFENSE_ASSAULT ON FEMALE,OFFENSE_ASSAULT ON OFFICER/ST EMPLOYEE,OFFENSE_ASSAULT ON POLICEMAN,OFFENSE_ASSAULT ON PUBLIC OFFICIAL,OFFENSE_AWDW,OFFENSE_AWDW GOV OFFICERS/EMPLOYEES,OFFENSE_AWDWISI,OFFENSE_AWDWWITK,OFFENSE_AWDWWITKISI,OFFENSE_B & E & L,OFFENSE_B & E VEHICLES,OFFENSE_BURGLARY 1ST DEGREE,OFFENSE_BURGLARY 2ND DEGREE,OFFENSE_CARRY CONCEALED WEAPON,OFFENSE_CHEAT - PROPERTY/SERVICES,OFFENSE_COMMON LAW FORGERY,OFFENSE_COMMON LAW ROBBERY,OFFENSE_COMMUNICATING THREATS,OFFENSE_CREDIT CARD THEFT,OFFENSE_CRIME AGAINST NATURE,OFFENSE_DAMAGE TO PROPERTY,OFFENSE_DEL/SELL SCHEDULE II,OFFENSE_DISCHG FIREARM-OCC PROPERTY,OFFENSE_DISORDERLY CONDUCT,OFFENSE_DRIV LICENSE PERM RVK,OFFENSE_DRIV LICENSE REVOKED,OFFENSE_DRIVING UNDER INFLUENCE (DUI),OFFENSE_DRUG PARA - USE/POSSESS,OFFENSE_DRUNK & DISORDERLY,OFFENSE_DWI DRIVING WHILE IMPAIRED,OFFENSE_DWI LEVEL 1,OFFENSE_DWI LEVEL 1 AGGRAVATED,OFFENSE_DWI LEVEL 2,OFFENSE_DWI LEVEL 3,OFFENSE_DWI LEVEL 4,OFFENSE_DWI LEVEL 5,OFFENSE_EMBEZZLEMENT,OFFENSE_EMPLOY SEC VIOLATION,OFFENSE_ESCAPE PRISON,OFFENSE_FAIL TO REGISTER (SEX OFFENDER,OFFENSE_FAILURE TO STOP FOR ACCIDENT,OFFENSE_FELONY B&E,OFFENSE_FINANCIAL CARD FRAUD/MISD,OFFENSE_FORGERY,OFFENSE_FORGERY AND UTTERING,OFFENSE_HABITUAL FELON,OFFENSE_HABITUAL IMPAIRED DRIVING,OFFENSE_HABITUAL MISDEMEANOR ASSAULT,OFFENSE_HIT AND RUN,OFFENSE_IDENTITY FRAUD/THEFT,OFFENSE_INDECENT LIBERTY W/CHILD,OFFENSE_INVOLUNTARY MANSLAUGHTER,OFFENSE_KIDNAPPING 1ST DEGREE,OFFENSE_KIDNAPPING 2ND DEGREE,OFFENSE_LARCENY,OFFENSE_LARCENY (OVER $200),OFFENSE_LARCENY AFTER B & E,OFFENSE_LARCENY AND RECEIVING,OFFENSE_LARCENY BY SERVANT/EMPLOYEE,OFFENSE_LARCENY FROM PERSON,OFFENSE_LARCENY OF FIREARMS,OFFENSE_LARCENY OF MOTOR VEHICLE,OFFENSE_LARCENY OVER $1000,OFFENSE_MAINT ANY PLACE CONTR SUB,OFFENSE_MANSLAUGHTER,OFFENSE_MISD B&E,OFFENSE_MURDER FIRST DEGREE,OFFENSE_MURDER SECOND DEGREE,OFFENSE_NON-SUPPORT,OFFENSE_OBSTRUCTING JUSTICE,OFFENSE_OBT PROP BY FALSE PR/CHTS/SER,OFFENSE_OBTAIN CONTR SUBST BY FRAUD,OFFENSE_OPERATE VEHICLE W/O LICENSE,OFFENSE_OTHER,OFFENSE_OTHER MISDEMEANANT,OFFENSE_POSSESS SCHEDULE I,OFFENSE_POSSESS SCHEDULE II,OFFENSE_POSSESS SCHEDULE VI,OFFENSE_POSSESS WIT SELL CONTROL SUBST,OFFENSE_POSSESS WITS SCHEDULE I,OFFENSE_POSSESS WITS SCHEDULE II,OFFENSE_POSSESS WITS SCHEDULE VI,OFFENSE_POSSESSING STOLEN GOODS,OFFENSE_POSSESSION OF FIREARM BY FELON,OFFENSE_POST RELEASE REVOCATION,OFFENSE_RAPE FIRST DEGREE,OFFENSE_RAPE SECOND DEGREE,OFFENSE_RECEIVING STOLEN GOODS,OFFENSE_RECEIVING STOLEN VEHICLE,OFFENSE_RECKLESS DRIVING,OFFENSE_RESISTING OFFICER,OFFENSE_ROBBERY W/DANGEROUS WEAPON,OFFENSE_SECOND DEGREE TRESPASS,OFFENSE_SELL CONTROL SUBSTANCE,OFFENSE_SELL SCHEDULE I,OFFENSE_SELL SCHEDULE II,OFFENSE_SELL SCHEDULE VI,OFFENSE_SEXUAL OFFENSE 1ST DEGREE,OFFENSE_SEXUAL OFFENSE 2ND DEGREE,OFFENSE_SHOPLIFTING,OFFENSE_SIMPLE ASSAULT/AFFRAY,OFFENSE_SPEED ELUDE ARREST/ATTEMPT/SEC,OFFENSE_SPEEDING,OFFENSE_SPEEDING ELUDE ARREST OR/ATTEM,OFFENSE_TRAFFICKING SCHEDULE I,OFFENSE_TRAFFICKING SCHEDULE II,OFFENSE_TRESPASS,OFFENSE_UNAUTH USE MOTOR CONVEYANCE,OFFENSE_UTTERING FORGEDPAPER/INST/END,OFFENSE_VIOLATE REGULATE CONTROL SUBST,OFFENSE_VIOLATE VEHICLE REGISTRATION,OFFENSE_VIOLATION CONT SUB PENAL INST,OFFENSE_VIOLATION DRUG LAWS,OFFENSE_VIOLATION PROTECTIVE ORDER,OFFENSE_WANTON INJ PER/PROP GT $200,OFFENSE_WILL/WANT INJ REAL PROPERTY,OFFENSE_WORTHLESS CHECK,FELON,MISD.,MAX.TERM:,MIN.TERM:,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,PRIOR_RCD._POINTS/CONVICTIONS,SENTENCE_START,DISCI._SEGREGATION_TIME_(DAYS),INFRAC_COUNT,INFRAC_ACTIVE RIO

In [216]:
dt_to_timestamp = lambda x: pd.to_datetime(x, errors='coerce')

In [131]:
final_data_no_preprocess.shape

(854066, 263)

### Overwrite all dates with year 2622 or later (pandas/ numpy max year)

In [218]:
final_data.loc[final_data['SENTENCE_END'] >= datetime.strptime('2261-12-31', '%Y-%m-%d'), 'AGE_AT_RELEASE'] =  pd.np.nan


In [ ]:
final_data.loc[final_data['SENTENCE_END'] >= datetime.strptime('2261-12-31', '%Y-%m-%d'), 'SENTENCE_END'] =  datetime.strptime('2230-01-01', '%Y-%m-%d')


In [ ]:
final_data = final_data.loc[final_data['SENTENCE_END'] <= datetime.strptime('2261-12-31', '%Y-%m-%d')]

### Convert date columns to Timestamp for Postgres Conversion

In [219]:
final_data.loc[:,'SENTENCE_END'] = final_data['SENTENCE_END'].apply(lambda x: dt_to_timestamp(x))


In [ ]:
final_data.loc[:,'INMATE_BIRTH_DATE'] = final_data['INMATE_BIRTH_DATE'].apply(lambda x: dt_to_timestamp(x))


In [ ]:
final_data.loc[:,'SENTENCE_START'] = final_data['SENTENCE_START'].apply(lambda x: dt_to_timestamp(x))


In [ ]:
final_data.index.names = ['record_id']

In [ ]:
final_data.head()

In [242]:
final_data.SENTENCE_END.dtype

dtype('<M8[ns]')

In [142]:
final_data_no_preprocess.to_pickle('data/preprocessed/final_recid_no_preprocess.pkl')

### Collapse County Indicator Columns for Triage

In [133]:
# confirm one county per row
# final_data_no_preprocess[final_data_no_preprocess.columns[final_data_no_preprocess.columns.str.contains('COUNTY')].tolist()].sum(axis=1).max()

1.0000000000000002

In [134]:
# conviction_county = pd.DataFrame(final_data_no_preprocess[final_data_no_preprocess.columns[final_data_no_preprocess.columns.str.contains('COUNTY')].tolist()].idxmax(axis=1))

In [135]:
# conviction_county.columns = ['COUNTY_FLAG']

In [136]:
# conviction_county.loc[:,'COUNTY_OF_CONVICTION_CODE'] = conviction_county['COUNTY_FLAG'].str.split('COUNTY_', n=1).str.join('')

In [138]:
# final_data_no_preprocess.drop(final_data_no_preprocess.columns[final_data_no_preprocess.columns.str.contains('COUNTY')].tolist(), axis=1).shape

(854066, 205)

In [140]:
# final_data_no_preprocess = final_data_no_preprocess.drop(final_data_no_preprocess.columns[final_data_no_preprocess.columns.str.contains('COUNTY')].tolist(), axis=1)

In [141]:
# final_data_no_preprocess = final_data_no_preprocess.merge(conviction_county['COUNTY_OF_CONVICTION_CODE'], left_index=True, right_index=True, how='left')

In [143]:
final_data = pd.read_pickle('data/preprocessed/final_recid_no_preprocess.pkl')

In [13]:
import os
os.getcwd()

'/mnt/data/users/lhinkson/triage/example/north_carolina'

### Get Column Names and Types for Table Creation

In [12]:
# final_data.to_pickle('data/preprocessed/final_recid_date_conv.pkl')
final_recid_date_conv = pd.read_pickle('data/preprocessed/final_recid_date_conv.pkl')

In [13]:
final_recid_date_conv.head()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_END,PAROLE_DAYS,NEW_PERIOD_OF_INCARCERATION_FL,P&P_COMMITMENT_STATUS_FLAG,NEW_PERIOD_OF_SUPERVISION_FLAG,INMATE_GENDER_CODE,INMATE_RACE_CODE,INMATE_BIRTH_DATE,AGE_AT_RELEASE,NUMBER_OF_COUNTS,OFFENSE_3M REIMPRISON VIOL,OFFENSE_ABANDONMENT,OFFENSE_ARMED ROBBERY,OFFENSE_ASSAULT,OFFENSE_ASSAULT BY STRANGULATION,OFFENSE_ASSAULT INFLICT SERI BODY INJ,OFFENSE_ASSAULT ISI,OFFENSE_ASSAULT ON FEMALE,OFFENSE_ASSAULT ON OFFICER/ST EMPLOYEE,OFFENSE_ASSAULT ON POLICEMAN,OFFENSE_ASSAULT ON PUBLIC OFFICIAL,OFFENSE_AWDW,OFFENSE_AWDW GOV OFFICERS/EMPLOYEES,OFFENSE_AWDWISI,OFFENSE_AWDWWITK,OFFENSE_AWDWWITKISI,OFFENSE_B & E & L,OFFENSE_B & E VEHICLES,OFFENSE_BURGLARY 1ST DEGREE,OFFENSE_BURGLARY 2ND DEGREE,OFFENSE_CARRY CONCEALED WEAPON,OFFENSE_CHEAT - PROPERTY/SERVICES,OFFENSE_COMMON LAW FORGERY,OFFENSE_COMMON LAW ROBBERY,OFFENSE_COMMUNICATING THREATS,OFFENSE_CREDIT CARD THEFT,OFFENSE_CRIME AGAINST NATURE,OFFENSE_DAMAGE TO PROPERTY,OFFENSE_DEL/SELL SCHEDULE II,OFFENSE_DISCHG FIREARM-OCC PROPERTY,OFFENSE_DISORDERLY CONDUCT,OFFENSE_DRIV LICENSE PERM RVK,OFFENSE_DRIV LICENSE REVOKED,OFFENSE_DRIVING UNDER INFLUENCE (DUI),OFFENSE_DRUG PARA - USE/POSSESS,OFFENSE_DRUNK & DISORDERLY,OFFENSE_DWI DRIVING WHILE IMPAIRED,OFFENSE_DWI LEVEL 1,OFFENSE_DWI LEVEL 1 AGGRAVATED,OFFENSE_DWI LEVEL 2,OFFENSE_DWI LEVEL 3,OFFENSE_DWI LEVEL 4,OFFENSE_DWI LEVEL 5,OFFENSE_EMBEZZLEMENT,OFFENSE_EMPLOY SEC VIOLATION,OFFENSE_ESCAPE PRISON,OFFENSE_FAIL TO REGISTER (SEX OFFENDER,OFFENSE_FAILURE TO STOP FOR ACCIDENT,OFFENSE_FELONY B&E,OFFENSE_FINANCIAL CARD FRAUD/MISD,OFFENSE_FORGERY,OFFENSE_FORGERY AND UTTERING,OFFENSE_HABITUAL FELON,OFFENSE_HABITUAL IMPAIRED DRIVING,OFFENSE_HABITUAL MISDEMEANOR ASSAULT,OFFENSE_HIT AND RUN,OFFENSE_IDENTITY FRAUD/THEFT,OFFENSE_INDECENT LIBERTY W/CHILD,OFFENSE_INVOLUNTARY MANSLAUGHTER,OFFENSE_KIDNAPPING 1ST DEGREE,OFFENSE_KIDNAPPING 2ND DEGREE,OFFENSE_LARCENY,OFFENSE_LARCENY (OVER $200),OFFENSE_LARCENY AFTER B & E,OFFENSE_LARCENY AND RECEIVING,OFFENSE_LARCENY BY SERVANT/EMPLOYEE,OFFENSE_LARCENY FROM PERSON,OFFENSE_LARCENY OF FIREARMS,OFFENSE_LARCENY OF MOTOR VEHICLE,OFFENSE_LARCENY OVER $1000,OFFENSE_MAINT ANY PLACE CONTR SUB,OFFENSE_MANSLAUGHTER,OFFENSE_MISD B&E,OFFENSE_MURDER FIRST DEGREE,OFFENSE_MURDER SECOND DEGREE,OFFENSE_NON-SUPPORT,OFFENSE_OBSTRUCTING JUSTICE,OFFENSE_OBT PROP BY FALSE PR/CHTS/SER,OFFENSE_OBTAIN CONTR SUBST BY FRAUD,OFFENSE_OPERATE VEHICLE W/O LICENSE,OFFENSE_OTHER,OFFENSE_OTHER MISDEMEANANT,OFFENSE_POSSESS SCHEDULE I,OFFENSE_POSSESS SCHEDULE II,OFFENSE_POSSESS SCHEDULE VI,OFFENSE_POSSESS WIT SELL CONTROL SUBST,OFFENSE_POSSESS WITS SCHEDULE I,OFFENSE_POSSESS WITS SCHEDULE II,OFFENSE_POSSESS WITS SCHEDULE VI,OFFENSE_POSSESSING STOLEN GOODS,OFFENSE_POSSESSION OF FIREARM BY FELON,OFFENSE_POST RELEASE REVOCATION,OFFENSE_RAPE FIRST DEGREE,OFFENSE_RAPE SECOND DEGREE,OFFENSE_RECEIVING STOLEN GOODS,OFFENSE_RECEIVING STOLEN VEHICLE,OFFENSE_RECKLESS DRIVING,OFFENSE_RESISTING OFFICER,OFFENSE_ROBBERY W/DANGEROUS WEAPON,OFFENSE_SECOND DEGREE TRESPASS,OFFENSE_SELL CONTROL SUBSTANCE,OFFENSE_SELL SCHEDULE I,OFFENSE_SELL SCHEDULE II,OFFENSE_SELL SCHEDULE VI,OFFENSE_SEXUAL OFFENSE 1ST DEGREE,OFFENSE_SEXUAL OFFENSE 2ND DEGREE,OFFENSE_SHOPLIFTING,OFFENSE_SIMPLE ASSAULT/AFFRAY,OFFENSE_SPEED ELUDE ARREST/ATTEMPT/SEC,OFFENSE_SPEEDING,OFFENSE_SPEEDING ELUDE ARREST OR/ATTEM,OFFENSE_TRAFFICKING SCHEDULE I,OFFENSE_TRAFFICKING SCHEDULE II,OFFENSE_TRESPASS,OFFENSE_UNAUTH USE MOTOR CONVEYANCE,OFFENSE_UTTERING FORGEDPAPER/INST/END,OFFENSE_VIOLATE REGULATE CONTROL SUBST,OFFENSE_VIOLATE VEHICLE REGISTRATION,OFFENSE_VIOLATION CONT SUB PENAL INST,OFFENSE_VIOLATION DRUG LAWS,OFFENSE_VIOLATION PROTECTIVE ORDER,OFFENSE_WANTON INJ PER/PROP GT $200,OFFENSE_WILL/WANT INJ REAL PROPERTY,OFFENSE_WORTHLESS CHECK,FELON,MISD.,MAX.TERM:,MIN.TERM:,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,PRIOR_RCD._POINTS/CONVICTIONS,SENTENCE_START,DISCI._SEGREGATION_TIME_(DAYS),INFRAC_COUNT,INFRAC_ACTIVE RIO

In [105]:
type_convsersion = {'float64': 'decimal', 'int64':'int', 'object': 'date'}

In [108]:
for col in final_data.columns: 
    print('_'.join(col.split()), type_convsersion[f"{final_data[col].dtype}"], ',')

INMATE_DOC_NUMBER int ,
INMATE_COMMITMENT_PREFIX date ,
SENTENCE_END date ,
PAROLE_DAYS decimal ,
NEW_PERIOD_OF_INCARCERATION_FL date ,
P&P_COMMITMENT_STATUS_FLAG date ,
NEW_PERIOD_OF_SUPERVISION_FLAG date ,
INMATE_GENDER_CODE date ,
INMATE_RACE_CODE date ,
INMATE_BIRTH_DATE date ,
AGE_AT_RELEASE int ,
NUMBER_OF_COUNTS int ,
OFFENSE_3M_REIMPRISON_VIOL decimal ,
OFFENSE_ABANDONMENT decimal ,
OFFENSE_ARMED_ROBBERY decimal ,
OFFENSE_ASSAULT decimal ,
OFFENSE_ASSAULT_BY_STRANGULATION decimal ,
OFFENSE_ASSAULT_INFLICT_SERI_BODY_INJ decimal ,
OFFENSE_ASSAULT_ISI decimal ,
OFFENSE_ASSAULT_ON_FEMALE decimal ,
OFFENSE_ASSAULT_ON_OFFICER/ST_EMPLOYEE decimal ,
OFFENSE_ASSAULT_ON_POLICEMAN decimal ,
OFFENSE_ASSAULT_ON_PUBLIC_OFFICIAL decimal ,
OFFENSE_AWDW decimal ,
OFFENSE_AWDW_GOV_OFFICERS/EMPLOYEES decimal ,
OFFENSE_AWDWISI decimal ,
OFFENSE_AWDWWITK decimal ,
OFFENSE_AWDWWITKISI decimal ,
OFFENSE_B_&_E_&_L decimal ,
OFFENSE_B_&_E_VEHICLES decimal ,
OFFENSE_BURGLARY_1ST_DEGREE decimal ,
OFFENSE

In [17]:
db_cols = [
'INMATE_DOC_NUMBER',
'INMATE_COMMITMENT_PREFIX ',
'SENTENCE_END ',
'PAROLE_DAYS ',
'NEW_PERIOD_OF_INCARCERATION_FL ',
'PP_COMMITMENT_STATUS_FLAG ',
'NEW_PERIOD_OF_SUPERVISION_FLAG ',
'INMATE_GENDER_CODE ',
'INMATE_RACE_CODE ',
'INMATE_BIRTH_DATE ',
'AGE_AT_RELEASE ',
'NUMBER_OF_COUNTS ',
'OFFENSE_3M_REIMPRISON_VIOL ',
'OFFENSE_ABANDONMENT ',
'OFFENSE_ARMED_ROBBERY ',
'OFFENSE_ASSAULT ',
'OFFENSE_ASSAULT_BY_STRANGULATION ',
'OFFENSE_ASSAULT_INFLICT_SERI_BODY_INJ ',
'OFFENSE_ASSAULT_ISI ',
'OFFENSE_ASSAULT_ON_FEMALE ',
'OFFENSE_ASSAULT_ON_OFFICER_ST_EMPLOYEE ',
'OFFENSE_ASSAULT_ON_POLICEMAN ',
'OFFENSE_ASSAULT_ON_PUBLIC_OFFICIAL ',
'OFFENSE_AWDW ',
'OFFENSE_AWDW_GOV_OFFICERS_EMPLOYEES ',
'OFFENSE_AWDWISI ',
'OFFENSE_AWDWWITK ',
'OFFENSE_AWDWWITKISI ',
'OFFENSE_B_E_L ',
'OFFENSE_B_E_VEHICLES ',
'OFFENSE_BURGLARY_1ST_DEGREE ',
'OFFENSE_BURGLARY_2ND_DEGREE ',
'OFFENSE_CARRY_CONCEALED_WEAPON ',
'OFFENSE_CHEAT_PROPERTY_SERVICES ',
'OFFENSE_COMMON_LAW_FORGERY ',
'OFFENSE_COMMON_LAW_ROBBERY ',
'OFFENSE_COMMUNICATING_THREATS ',
'OFFENSE_CREDIT_CARD_THEFT ',
'OFFENSE_CRIME_AGAINST_NATURE ',
'OFFENSE_DAMAGE_TO_PROPERTY ',
'OFFENSE_DEL_SELL_SCHEDULE_II ',
'OFFENSE_DISCHG_FIREARM_OCC_PROPERTY ',
'OFFENSE_DISORDERLY_CONDUCT ',
'OFFENSE_DRIV_LICENSE_PERM_RVK ',
'OFFENSE_DRIV_LICENSE_REVOKED ',
'OFFENSE_DRIVING_UNDER_INFLUENCE_DUI ',
'OFFENSE_DRUG_PARA_USE_POSSESS ',
'OFFENSE_DRUNK_DISORDERLY ',
'OFFENSE_DWI_DRIVING_WHILE_IMPAIRED ',
'OFFENSE_DWI_LEVEL_1 ',
'OFFENSE_DWI_LEVEL_1_AGGRAVATED ',
'OFFENSE_DWI_LEVEL_2 ',
'OFFENSE_DWI_LEVEL_3 ',
'OFFENSE_DWI_LEVEL_4 ',
'OFFENSE_DWI_LEVEL_5 ',
'OFFENSE_EMBEZZLEMENT ',
'OFFENSE_EMPLOY_SEC_VIOLATION ',
'OFFENSE_ESCAPE_PRISON ',
'OFFENSE_FAIL_TO_REGISTER_SEX_OFFENDER ',
'OFFENSE_FAILURE_TO_STOP_FOR_ACCIDENT ',
'OFFENSE_FELONY_B_E ',
'OFFENSE_FINANCIAL_CARD_FRAUD_MISD ',
'OFFENSE_FORGERY ',
'OFFENSE_FORGERY_AND_UTTERING ',
'OFFENSE_HABITUAL_FELON ',
'OFFENSE_HABITUAL_IMPAIRED_DRIVING ',
'OFFENSE_HABITUAL_MISDEMEANOR_ASSAULT ',
'OFFENSE_HIT_AND_RUN ',
'OFFENSE_IDENTITY_FRAUD_THEFT ',
'OFFENSE_INDECENT_LIBERTY_W_CHILD ',
'OFFENSE_INVOLUNTARY_MANSLAUGHTER ',
'OFFENSE_KIDNAPPING_1ST_DEGREE ',
'OFFENSE_KIDNAPPING_2ND_DEGREE ',
'OFFENSE_LARCENY ',
'OFFENSE_LARCENY_OVER_200 ',
'OFFENSE_LARCENY_AFTER_B_E ',
'OFFENSE_LARCENY_AND_RECEIVING ',
'OFFENSE_LARCENY_BY_SERVANT_EMPLOYEE ',
'OFFENSE_LARCENY_FROM_PERSON ',
'OFFENSE_LARCENY_OF_FIREARMS ',
'OFFENSE_LARCENY_OF_MOTOR_VEHICLE ',
'OFFENSE_LARCENY_OVER_1000 ',
'OFFENSE_MAINT_ANY_PLACE_CONTR_SUB ',
'OFFENSE_MANSLAUGHTER ',
'OFFENSE_MISD_B_E ',
'OFFENSE_MURDER_FIRST_DEGREE ',
'OFFENSE_MURDER_SECOND_DEGREE ',
'OFFENSE_NON_SUPPORT ',
'OFFENSE_OBSTRUCTING_JUSTICE ',
'OFFENSE_OBT_PROP_BY_FALSE_PR_CHTS_SER ',
'OFFENSE_OBTAIN_CONTR_SUBST_BY_FRAUD ',
'OFFENSE_OPERATE_VEHICLE_WO_LICENSE ',
'OFFENSE_OTHER ',
'OFFENSE_OTHER_MISDEMEANANT ',
'OFFENSE_POSSESS_SCHEDULE_I ',
'OFFENSE_POSSESS_SCHEDULE_II ',
'OFFENSE_POSSESS_SCHEDULE_VI ',
'OFFENSE_POSSESS_WIT_SELL_CONTROL_SUBST ',
'OFFENSE_POSSESS_WITS_SCHEDULE_I ',
'OFFENSE_POSSESS_WITS_SCHEDULE_II ',
'OFFENSE_POSSESS_WITS_SCHEDULE_VI ',
'OFFENSE_POSSESSING_STOLEN_GOODS ',
'OFFENSE_POSSESSION_OF_FIREARM_BY_FELON ',
'OFFENSE_POST_RELEASE_REVOCATION ',
'OFFENSE_RAPE_FIRST_DEGREE ',
'OFFENSE_RAPE_SECOND_DEGREE ',
'OFFENSE_RECEIVING_STOLEN_GOODS ',
'OFFENSE_RECEIVING_STOLEN_VEHICLE ',
'OFFENSE_RECKLESS_DRIVING ',
'OFFENSE_RESISTING_OFFICER ',
'OFFENSE_ROBBERY_W_DANGEROUS_WEAPON ',
'OFFENSE_SECOND_DEGREE_TRESPASS ',
'OFFENSE_SELL_CONTROL_SUBSTANCE ',
'OFFENSE_SELL_SCHEDULE_I ',
'OFFENSE_SELL_SCHEDULE_II ',
'OFFENSE_SELL_SCHEDULE_VI ',
'OFFENSE_SEXUAL_OFFENSE_1ST_DEGREE ',
'OFFENSE_SEXUAL_OFFENSE_2ND_DEGREE ',
'OFFENSE_SHOPLIFTING ',
'OFFENSE_SIMPLE_ASSAULT_AFFRAY ',
'OFFENSE_SPEED_ELUDE_ARREST_ATTEMPT_SEC ',
'OFFENSE_SPEEDING ',
'OFFENSE_SPEEDING_ELUDE_ARREST_OR_ATTEM ',
'OFFENSE_TRAFFICKING_SCHEDULE_I ',
'OFFENSE_TRAFFICKING_SCHEDULE_II ',
'OFFENSE_TRESPASS ',
'OFFENSE_UNAUTH_USE_MOTOR_CONVEYANCE ',
'OFFENSE_UTTERING_FORGEDPAPER_INST_END ',
'OFFENSE_VIOLATE_REGULATE_CONTROL_SUBST ',
'OFFENSE_VIOLATE_VEHICLE_REGISTRATION ',
'OFFENSE_VIOLATION_CONT_SUB_PENAL_INST ',
'OFFENSE_VIOLATION_DRUG_LAWS ',
'OFFENSE_VIOLATION_PROTECTIVE_ORDER ',
'OFFENSE_WANTON_INJ_PER_PROP_GT_200 ',
'OFFENSE_WILL_WANT_INJ_REAL_PROPERTY ',
'OFFENSE_WORTHLESS_CHECK ',
'FELON ',
'MISD ',
'MAX_TERM ',
'MIN_TERM ',
'MINIMUM_SENTENCE_LENGTH ',
'MAXIMUM_SENTENCE_LENGTH ',
'LENGTH_OF_SUPERVISION ',
'PRIOR_RCD_POINTS_CONVICTIONS ',
'SENTENCE_START ',
'DISCI_SEGREGATION_TIME_DAYS ',
'INFRAC_COUNT ',
'INFRAC_ACTIVE_RIOTER ',
'INFRAC_ASSAULT_PERSON_W_WEAPON ',
'INFRAC_ASSAULT_STAFF_W_WEAPON ',
'INFRAC_ASSAULT_STAFF_THROWING_LIQUIDS ',
'INFRAC_ASSLT_OTHER_W_UNLIKELY_INJ ',
'INFRAC_ASSLT_STAFF_W_UNLIKELY_INJ ',
'INFRAC_ATTEMPT_CLASS_A_OFFENSE ',
'INFRAC_ATTEMPT_CLASS_B_OFFENSE ',
'INFRAC_ATTEMPT_CLASS_C_OFFENSE ',
'INFRAC_ATTEMPT_CLASS_D_OFFENSE ',
'INFRAC_BARTER_TRADE_LOAN_MONEY ',
'INFRAC_CREATE_OFFENSIVE_CONDITION ',
'INFRAC_DAMAGE_STATE_ANOTHERS_PROPERTY ',
'INFRAC_DISOBEY_ORDER ',
'INFRAC_ESCAPE ',
'INFRAC_FAKE_ILLNESS ',
'INFRAC_FALSE_ALLEGATIONS_ON_STAFF ',
'INFRAC_FIGHT_W_WEAPON_OR_REQ_OUT_MED ',
'INFRAC_FIGHTING ',
'INFRAC_FLOOD_CELL ',
'INFRAC_GAMBLING ',
'INFRAC_HIGH_RISK_ACT ',
'INFRAC_ILLEGAL_CLOTH_LINEN_SHEETS ',
'INFRAC_INTERFERE_W_STAFF ',
'INFRAC_INVOLVEMENT_W_GANG_OR_SRG ',
'INFRAC_LEAVE_QUIT_COMM_BASED_PROGRAM ',
'INFRAC_LOCK_TAMPERING ',
'INFRAC_MISUSE_MEDICINE ',
'INFRAC_MISUSE_SUPPLIES ',
'INFRAC_MISUSE_UNAUTH_USE_PHONE_MAIL ',
'INFRAC_NEGLIGENTLY_PERFORM_DUTIES ',
'INFRAC_NO_THREAT_CONTRABAND ',
'INFRAC_OFFER_ACCEPT_BRIBE_STAFF ',
'INFRAC_OTHER ',
'INFRAC_POSS_AUDIO_VIDEO_IMAGE_DEVICE ',
'INFRAC_POSS_MONEY_UNAUTHORIZED_FUNDS ',
'INFRAC_POSSESS_EXCESS_STAMPS ',
'INFRAC_PROFANE_LANGUAGE ',
'INFRAC_PROPERTY_TAMPERING ',
'INFRAC_PROVOKE_ASSAULT ',
'INFRAC_REFUSE_SUBMIT_DRUG_BREATH_TEST ',
'INFRAC_SELF_INJURY ',
'INFRAC_SELL_MISUSE_MEDICATION ',
'INFRAC_SET_A_FIRE ',
'INFRAC_SEXUAL_ACT ',
'INFRAC_SUBSTANCE_POSSESSION ',
'INFRAC_THEFT_OF_PROPERTY ',
'INFRAC_THREATEN_TO_HARM_INJURE_STAFF ',
'INFRAC_UNAUTH_TOBACCO_NON_PERSNL_USE ',
'INFRAC_UNAUTHORIZED_FUNDS ',
'INFRAC_UNAUTHORIZED_LEAVE ',
'INFRAC_UNAUTHORIZED_LOCATION ',
'INFRAC_UNAUTHORIZED_TOBACCO_USE ',
'INFRAC_UNKEMPT_ROOM ',
'INFRAC_VERBAL_THREAT ',
'INFRAC_WEAPON_POSSESSION ',
'PREVIOUS_COMMITMENTS ',
'RECIDIVATED ',
'COUNTY_OF_CONVICTION_CODE']

In [18]:
db_cols_clean = [col.lower().strip() for col in db_cols]

In [19]:
db_cols_clean

['inmate_doc_number',
 'inmate_commitment_prefix',
 'sentence_end',
 'parole_days',
 'new_period_of_incarceration_fl',
 'pp_commitment_status_flag',
 'new_period_of_supervision_flag',
 'inmate_gender_code',
 'inmate_race_code',
 'inmate_birth_date',
 'age_at_release',
 'number_of_counts',
 'offense_3m_reimprison_viol',
 'offense_abandonment',
 'offense_armed_robbery',
 'offense_assault',
 'offense_assault_by_strangulation',
 'offense_assault_inflict_seri_body_inj',
 'offense_assault_isi',
 'offense_assault_on_female',
 'offense_assault_on_officer_st_employee',
 'offense_assault_on_policeman',
 'offense_assault_on_public_official',
 'offense_awdw',
 'offense_awdw_gov_officers_employees',
 'offense_awdwisi',
 'offense_awdwwitk',
 'offense_awdwwitkisi',
 'offense_b_e_l',
 'offense_b_e_vehicles',
 'offense_burglary_1st_degree',
 'offense_burglary_2nd_degree',
 'offense_carry_concealed_weapon',
 'offense_cheat_property_services',
 'offense_common_law_forgery',
 'offense_common_law_robbery',

In [20]:
df_to_db_name_mapping = {df_col: db_col for df_col, db_col in zip(final_recid_date_conv.columns, db_cols_clean)}

In [21]:
df_to_db_name_mapping

{'INMATE_DOC_NUMBER': 'inmate_doc_number',
 'INMATE_COMMITMENT_PREFIX': 'inmate_commitment_prefix',
 'SENTENCE_END': 'sentence_end',
 'PAROLE_DAYS': 'parole_days',
 'NEW_PERIOD_OF_INCARCERATION_FL': 'new_period_of_incarceration_fl',
 'P&P_COMMITMENT_STATUS_FLAG': 'pp_commitment_status_flag',
 'NEW_PERIOD_OF_SUPERVISION_FLAG': 'new_period_of_supervision_flag',
 'INMATE_GENDER_CODE': 'inmate_gender_code',
 'INMATE_RACE_CODE': 'inmate_race_code',
 'INMATE_BIRTH_DATE': 'inmate_birth_date',
 'AGE_AT_RELEASE': 'age_at_release',
 'NUMBER_OF_COUNTS': 'number_of_counts',
 'OFFENSE_3M REIMPRISON VIOL': 'offense_3m_reimprison_viol',
 'OFFENSE_ABANDONMENT': 'offense_abandonment',
 'OFFENSE_ARMED ROBBERY': 'offense_armed_robbery',
 'OFFENSE_ASSAULT': 'offense_assault',
 'OFFENSE_ASSAULT BY STRANGULATION': 'offense_assault_by_strangulation',
 'OFFENSE_ASSAULT INFLICT SERI BODY INJ': 'offense_assault_inflict_seri_body_inj',
 'OFFENSE_ASSAULT ISI': 'offense_assault_isi',
 'OFFENSE_ASSAULT ON FEMALE': 

In [23]:
final_recid_date_conv.rename(df_to_db_name_mapping, axis=1)

,inmate_doc_number,inmate_commitment_prefix,sentence_end,parole_days,new_period_of_incarceration_fl,pp_commitment_status_flag,new_period_of_supervision_flag,inmate_gender_code,inmate_race_code,inmate_birth_date,age_at_release,number_of_counts,offense_3m_reimprison_viol,offense_abandonment,offense_armed_robbery,offense_assault,offense_assault_by_strangulation,offense_assault_inflict_seri_body_inj,offense_assault_isi,offense_assault_on_female,offense_assault_on_officer_st_employee,offense_assault_on_policeman,offense_assault_on_public_official,offense_awdw,offense_awdw_gov_officers_employees,offense_awdwisi,offense_awdwwitk,offense_awdwwitkisi,offense_b_e_l,offense_b_e_vehicles,offense_burglary_1st_degree,offense_burglary_2nd_degree,offense_carry_concealed_weapon,offense_cheat_property_services,offense_common_law_forgery,offense_common_law_robbery,offense_communicating_threats,offense_credit_card_theft,offense_crime_against_nature,offense_damage_to_property,offense_del_sell_schedule_ii,offense_dischg_firearm_occ_property,offense_disorderly_conduct,offense_driv_license_perm_rvk,offense_driv_license_revoked,offense_driving_under_influence_dui,offense_drug_para_use_possess,offense_drunk_disorderly,offense_dwi_driving_while_impaired,offense_dwi_level_1,offense_dwi_level_1_aggravated,offense_dwi_level_2,offense_dwi_level_3,offense_dwi_level_4,offense_dwi_level_5,offense_embezzlement,offense_employ_sec_violation,offense_escape_prison,offense_fail_to_register_sex_offender,offense_failure_to_stop_for_accident,offense_felony_b_e,offense_financial_card_fraud_misd,offense_forgery,offense_forgery_and_uttering,offense_habitual_felon,offense_habitual_impaired_driving,offense_habitual_misdemeanor_assault,offense_hit_and_run,offense_identity_fraud_theft,offense_indecent_liberty_w_child,offense_involuntary_manslaughter,offense_kidnapping_1st_degree,offense_kidnapping_2nd_degree,offense_larceny,offense_larceny_over_200,offense_larceny_after_b_e,offense_larceny_and_receiving,offense_larceny_by_servant_employee,offense_larceny_from_person,offense_larceny_of_firearms,offense_larceny_of_motor_vehicle,offense_larceny_over_1000,offense_maint_any_place_contr_sub,offense_manslaughter,offense_misd_b_e,offense_murder_first_degree,offense_murder_second_degree,offense_non_support,offense_obstructing_justice,offense_obt_prop_by_false_pr_chts_ser,offense_obtain_contr_subst_by_fraud,offense_operate_vehicle_wo_license,offense_other,offense_other_misdemeanant,offense_possess_schedule_i,offense_possess_schedule_ii,offense_possess_schedule_vi,offense_possess_wit_sell_control_subst,offense_possess_wits_schedule_i,offense_possess_wits_schedule_ii,offense_possess_wits_schedule_vi,offense_possessing_stolen_goods,offense_possession_of_firearm_by_felon,offense_post_release_revocation,offense_rape_first_degree,offense_rape_second_degree,offense_receiving_stolen_goods,offense_receiving_stolen_vehicle,offense_reckless_driving,offense_resisting_officer,offense_robbery_w_dangerous_weapon,offense_second_degree_trespass,offense_sell_control_substance,offense_sell_schedule_i,offense_sell_schedule_ii,offense_sell_schedule_vi,offense_sexual_offense_1st_degree,offense_sexual_offense_2nd_degree,offense_shoplifting,offense_simple_assault_affray,offense_speed_elude_arrest_attempt_sec,offense_speeding,offense_speeding_elude_arrest_or_attem,offense_trafficking_schedule_i,offense_trafficking_schedule_ii,offense_trespass,offense_unauth_use_motor_conveyance,offense_uttering_forgedpaper_inst_end,offense_violate_regulate_control_subst,offense_violate_vehicle_registration,offense_violation_cont_sub_penal_inst,offense_violation_drug_laws,offense_violation_protective_order,offense_wanton_inj_per_prop_gt_200,offense_will_want_inj_real_property,offense_worthless_check,felon,misd,max_term,min_term,minimum_sentence_length,maximum_sentence_length,length_of_supervision,prior_rcd_points_convictions,sentence_start,disci_segregation_time_days,infrac_count,infrac_active_rioter,infrac_assault_person_w_wea

In [24]:
final_recid_date_conv_db_names = final_recid_date_conv.rename(df_to_db_name_mapping, axis=1)

In [25]:
# final_recid_date_conv_db_names.to_pickle('data/preprocessed/final_recid_date_conv_db_names.pkl')

In [ ]:
final_recid_date_conv_db_names = pd.read_pickle('data/preprocessed/final_recid_date_conv_db_names.pkl')